In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
funding = pd.read_excel('../WorkBook2.xlsx', skiprows=3)

In [3]:
funding

,Year,State,CoC Name,Org Name,Program Name,Program Type,Award Amount
0,2021,AK,NaN,Alaska,CDBG,NaN,3348442.0
1,2021,AK,NaN,"Anchorage, AK",CDBG,NaN,1948478.0
2,2021,AL,NaN,Alabama,CDBG,NaN,24256102.0
3,2021,AL,NaN,"Anniston, AL",CDBG,NaN,563039.0
4,2021,AL,NaN,"Auburn, AL",CDBG,NaN,686092.0
...,...,...,...,...,...,...,...
25598,2003,WV,NaN,West Virginia,CDBG,NaN,20364000.0
25599,2003,WV,NaN,"Wheeling, WV",CDBG,NaN,1820000.0
25600,2003,WY,NaN,"Casper, WY",CDBG,NaN,551000.0
25601,2003,WY,NaN,"Cheyenne, WY",CDBG,NaN,670000.0


In [4]:
#merged_df2.columns

In [5]:
population_est = pd.read_csv('../co-est2020-alldata.csv', encoding = "ISO-8859-1")
#popest.head()

In [6]:
population_est.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019,RNETMIG2020
0,40,3,6,1,0,Alabama,Alabama,4779736,4780118,4785514,...,0.680010,1.282825,1.614391,0.653828,0.730419,0.854384,1.197150,1.966095,2.677866,3.091308
1,50,3,6,1,1,Alabama,Autauga County,54571,54582,54761,...,6.236931,-5.971016,-3.773344,2.206640,-1.529706,4.954403,0.993228,-0.018021,3.486011,6.290545
2,50,3,6,1,3,Alabama,Baldwin County,182265,182263,183121,...,16.705437,17.670696,22.924288,20.300088,17.902273,21.436499,22.476720,24.846335,25.242507,26.401562
3,50,3,6,1,5,Alabama,Barbour County,27457,27454,27325,...,0.329254,-6.860371,-8.093425,-5.063857,-15.677998,-18.377839,-25.138734,-8.790155,-6.257064,0.649799
4,50,3,6,1,7,Alabama,Bibb County,22915,22904,22858,...,-4.912927,-3.789130,-5.800695,1.420612,1.286202,-0.841769,-3.235672,-7.271592,0.268980,-7.199262


In [7]:
# Read in the data, there are some encoding issues so we need to explicitly specify the encoding.
_CAINC4__ALL_AREAS_1969_2020_df = pd.read_csv('CAINC4__ALL_AREAS_1969_2020.csv',low_memory=False, encoding = "ISO-8859-1" )

# Clean up the description field a bit.
_CAINC4__ALL_AREAS_1969_2020_df['Description'] = _CAINC4__ALL_AREAS_1969_2020_df.Description.str.strip().str.replace("/","")

# Seems like there is double data for 'Employer contributions for government social insurance' but they have different LineCodes so we can take out one of them.
# The other descriptions are in 30's and on so we will take out 62.
_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[~(_CAINC4__ALL_AREAS_1969_2020_df.LineCode == 62)]

In [8]:
#_CAINC4__ALL_AREAS_1969_2020_df

In [9]:
all_data_ei = pd.DataFrame()
for i in range(2009,2020):  
# for i in range(1969,1970):
    year = str(i)
    print(f'Working on year : {year}')
    temp_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[[year, 'GeoFIPS', 'GeoName', 'Description']].copy()
    pivot_CAINC4__ALL_AREAS_1969_2020_df = pd.pivot_table(temp_CAINC4__ALL_AREAS_1969_2020_df, 
                                                    values=year
                                                    ,index=['GeoFIPS','GeoName']
                                                    ,columns=['Description']
                                                    ,aggfunc=np.sum, fill_value=0)

    pivot_CAINC4__ALL_AREAS_1969_2020_df['year'] = year
    pivot_CAINC4__ALL_AREAS_1969_2020_df.reset_index(inplace=True)
    all_data_ei = pd.concat([all_data_ei, pivot_CAINC4__ALL_AREAS_1969_2020_df])  

Working on year : 2009
Working on year : 2010
Working on year : 2011
Working on year : 2012
Working on year : 2013
Working on year : 2014
Working on year : 2015
Working on year : 2016
Working on year : 2017
Working on year : 2018
Working on year : 2019


In [10]:
all_data_ei.head(20)

Description,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
0,"""00000""",United States,8688852000,505515000,1046866000,457312000,7729044000,56103000,32713000,962827000,...,2197670000,2146693000,306771529,37394700,943183000,1504178000,173636700,136242000,6241491000,2009
1,"""01000""",Alabama,104185812,6728491,13425623,5607071,93653903,902231,707931,12335562,...,27582774,34676799,4757938,511409,8846794,19032694,2479507,1968098,76306324,2009
2,"""01001""","Autauga, AL",535644,39092,69250,28256,1160479,11424,8731,67348,...,291014,334471,54135,5188,55039,97506,17483,12295,383099,2009
3,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,1483633,1255988,179406,24644,242733,478837,89324,64680,1999571,2009
4,"""01005""","Barbour, AL",429668,28956,64008,23435,387341,18712,15908,52391,...,117582,219382,27657,2724,39544,87443,12978,10254,302681,2009
5,"""01007""","Bibb, AL",206853,15973,29693,10729,321855,2655,2515,26702,...,61383,161295,22941,1753,22435,40422,6304,4551,143996,2009
6,"""01009""","Blount, AL",462562,36182,56706,22040,993044,20257,16640,58222,...,213733,377714,57341,4962,90915,78746,15000,10038,292901,2009
7,"""01011""","Bullock, AL",135903,8373,19881,6779,137654,16503,11354,15152,...,39517,84517,10987,779,21785,26660,4023,3244,87458,2009
8,"""01013""","Butler, AL",280481,19731,38052,14717,296005,14418,12441,34448,...,83009,194578,20867,2109,36662,52769,8816,6707,191050,2009
9,"""01015""","Calhoun, AL",2542957,161856,391410,138911,1984590,13065,8742,300767,...,636167,957511,118363,10292,147434,530321,61551,51259,1865202,2009


In [11]:
states = ["United States", "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
all_data_ei = all_data_ei[all_data_ei.GeoName.isin(states) == False]
#all_data_ei

In [12]:
all_data_ei.drop(all_data_ei.tail(8).index, inplace = True)
all_data_ei

Description,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
2,"""01001""","Autauga, AL",535644,39092,69250,28256,1160479,11424,8731,67348,...,291014,334471,54135,5188,55039,97506,17483,12295,383099,2009
3,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,1483633,1255988,179406,24644,242733,478837,89324,64680,1999571,2009
4,"""01005""","Barbour, AL",429668,28956,64008,23435,387341,18712,15908,52391,...,117582,219382,27657,2724,39544,87443,12978,10254,302681,2009
5,"""01007""","Bibb, AL",206853,15973,29693,10729,321855,2655,2515,26702,...,61383,161295,22941,1753,22435,40422,6304,4551,143996,2009
6,"""01009""","Blount, AL",462562,36182,56706,22040,993044,20257,16640,58222,...,213733,377714,57341,4962,90915,78746,15000,10038,292901,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,"""56037""","Sweetwater, WY",2186822,122914,243513,125358,1778004,9846,8485,248272,...,302976,320098,42917,4162,361711,368871,27910,23748,1456240,2019
3188,"""56039""","Teton, WY",1858314,102201,137206,107871,1519128,3364,-483,210072,...,3548261,144956,23385,11783,406615,245077,34016,22233,1206622,2019
3189,"""56041""","Uinta, WY",546044,34219,79408,36393,536355,4720,2823,70612,...,133433,166160,20196,2965,42050,115801,11853,8888,388193,2019
3190,"""56043""","Washakie, WY",255080,15662,33784,15405,218093,8598,4675,31067,...,99229,82778,7824,1343,43774,49189,5091,3748,162117,2019


In [13]:
#remove all NAs
all_data_ei = all_data_ei.drop(all_data_ei[all_data_ei["Earnings by place of work"] == "(NA)"].index)
#checking
#all_data_ei[all_data_ei["Earnings by place of work"] == "(NA)"]

In [14]:
all_data_ei[all_data_ei["GeoName"].str.contains("+", regex=False)]

Description,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
588,"""15901""","Maui + Kalawao, HI*",3798997,230024,406004,193228,3383958,71658,4517,423252,...,1035278,866382,153393,25266,499744,599232,94943,69677,2700021,2009
2975,"""51901""","Albemarle + Charlottesville, VA*",5701735,336287,735728,281172,4027554,-402,-9261,617459,...,2093065,723620,141125,22159,646934,1016900,113544,91385,4037901,2009
2976,"""51903""","Alleghany + Covington, VA*",430036,30258,57504,24266,368274,-1641,-1879,54524,...,111292,211809,22254,1428,16811,81770,10110,8682,331455,2009
2977,"""51907""","Augusta, Staunton + Waynesboro, VA*",2592276,170405,322288,122199,2606758,12773,3683,292604,...,713753,836390,118593,13229,453984,444487,62262,49033,1693805,2009
2978,"""51911""","Campbell + Lynchburg, VA*",3665128,226917,462898,203912,2271715,-2272,-4341,430829,...,661497,1049419,129609,12130,152169,666810,85574,73444,2846149,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,"""51949""","Southampton + Franklin, VA*",445966,33079,65737,22172,591903,22538,18682,55251,...,212952,309731,25664,2500,43284,87909,11172,8672,314773,2019
2994,"""51951""","Spotsylvania + Fredericksburg, VA*",4121624,266832,453936,213163,5773418,2942,1573,479995,...,1496004,1329744,165701,19484,428859,667099,84803,65319,3025666,2019
2995,"""51953""","Washington + Bristol, VA*",1670965,124175,221479,88314,1428899,-703,-3802,212489,...,588026,830196,70888,8106,152990,309793,39157,31051,1208182,2019
2996,"""51955""","Wise + Norton, VA*",839644,65536,129250,45728,631798,-239,-338,111264,...,175647,561252,41410,2681,43386,174978,19026,16345,621280,2019


In [15]:
#Change names for smoother merging
combined_counties = {
    'Maui + Kalawao, HI*':'Maui, HI',
    'Albemarle + Charlottesville, VA*':'Albemarle, VA',
    'Alleghany + Covington, VA*':'Alleghany, VA',
    'Campbell + Lynchburg, VA*':'Campbell, VA',
    'Carroll + Galax, VA*':'Carroll, VA',
    'Frederick + Winchester, VA*':'Frederick, VA',
    'Greensville + Emporia, VA*':'Greensville, VA',
    'Henry + Martinsville, VA*':'Henry, VA',
    'James City + Williamsburg, VA*':'James City, VA',
    'Montgomery + Radford, VA*':'Montgomery, VA',
    'Pittsylvania + Danville, VA*':'Pittsylvania, VA',
    'Prince George + Hopewell, VA*':'Prince George, VA',
    'Roanoke + Salem, VA*':'Roanoke, VA',
    'Rockingham + Harrisonburg, VA*':'Rockingham, VA',
    'Southampton + Franklin, VA*':'Southampton, VA',
    'Spotsylvania + Fredericksburg, VA*':'Spotsylvania, VA',
    'Washington + Bristol, VA*':'Washington, VA',
    'Wise + Norton, VA*':'Wise, VA',
    'York + Poquoson, VA*':'York, VA',
    'Augusta, Staunton + Waynesboro, VA*':'Augusta, VA',
    'Fairfax, Fairfax City + Falls Church, VA*':'Fairfax, VA',
    'Rockbridge, Buena Vista + Lexington, VA*':'Rockbridge, VA',
    'Dinwiddie, Colonial Heights + Petersburg, VA*':'Dinwiddie, VA',
    'Prince William, Manassas + Manassas Park, VA*':'Prince William, VA'
}

independent_cities = {
    'Baltimore (Independent City), MD': 'Baltimore City, MD', 
    'Alexandria (Independent City), VA':'Alexandria City, VA',
    'Suffolk (Independent City), VA':'Suffolk City, VA',
    'Virginia Beach (Independent City), VA':'Virginia Beach City, VA',
    'Richmond (Independent City), VA':'Richmond City, VA',
    'Roanoke (Independent City), VA':'Roanoke City, VA',
    'Hampton (Independent City), VA':'Hampton City, VA',
    'Newport News (Independent City), VA':'Newport News City, VA',
    'Norfolk (Independent City), VA':'Norfolk City, VA',
    'Chesapeake (Independent City), VA':'Chesapeake City, VA',
    'Newport News (Independent City), VA':'Newport News City, VA',
    'Norfolk (Independent City), VA':'Norfolk City, VA',
    'Portsmouth (Independent City), VA':'Portsmouth City, VA',
    'St. Louis (Independent City), MO':'St. Louis City, MO',
    'Carson City (Independent City), NV':'Carson City, NV'
}

remove_asterisk = {
    "Aleutians East Borough, AK*":"Aleutians East Borough, AK",
    "Aleutians West Census Area, AK*": "Aleutians West Census Area, AK",
    "Aleutian Islands Census Area, AK*": "Aleutian Islands Census Area, AK",
    "Aleutians West Census Area, AK*":"Aleutians West Census Area, AK",
    "Bethel Census Area, AK*":"Bethel Census Area, AK",
    "Chugach Census Area, AK*":"Chugach Census Area, AK",
    "Copper River Census Area, AK*":"Copper River Census Area, AK",
    "Denali Borough, AK*":"Denali Borough, AK",
    "Dillingham Census Area, AK*":"Dillingham Census Area, AK",
    "Haines Borough, AK*":"Haines Borough, AK",
    "Hoonah-Angoon Census Area, AK*":"Hoonah-Angoon Census Area, AK",
    "Kenai Peninsula Borough, AK*":"Kenai Peninsula Borough, AK",
    "Ketchikan Gateway Borough, AK*":"Ketchikan Gateway Borough, AK",
    "Kusilvak Census Area, AK*":"Kusilvak Census Area, AK",
    "Lake and Peninsula Borough, AK*":"Lake and Peninsula Borough, AK",
    "North Slope Borough, AK*":"North Slope Borough, AK",
    "Prince of Wales-Hyder Census Area, AK*":"Prince of Wales-Hyder Census Area, AK",
    "Sitka City and Borough, AK*":"Sitka City and Borough, AK",
    "Skagway Municipality, AK*":"Skagway Municipality, AK",
    "Southeast Fairbanks Census Area, AK*":"Southeast Fairbanks Census Area, AK",
    "Valdez-Cordova Census Area, AK*":"Valdez-Cordova Census Area, AK",
    "La Paz, AZ*":"La Paz, AZ",
    "Petersburg Borough, AK*":"Petersburg Borough, AK",
    "Wrangell City and Borough, AK*":"Wrangell City and Borough, AK",
    "Yakutat City and Borough, AK*":"Yakutat City and Borough, AK",
    "Yukon-Koyukuk Census Area, AK*":"Yukon-Koyukuk Census Area, AK",
    "Yuma, AZ*":"Yuma, AZ",
    "Adams, CO*":"Adams, CO",
    "Boulder, CO*":"Boulder, CO",
    "Broomfield, CO*":"Broomfield, CO",
    "Jefferson, CO*":"Jefferson, CO",
    "Weld, CO*":"Weld, CO",
    "Cibola, NM*":"Cibola, NM",
    "Valencia, NM*":"Valencia, NM",
    "Oglala Lakota, SD*":"Oglala Lakota, SD",
    "Bedford, VA*":"Bedford, VA",
    "Menominee, WI*":"Menominee, WI",
    "Shawano, WI*":"Shawano, WI",
    "Fremont (includes Yellowstone Park), ID": "Fremont, ID",
    "Lagrange, IN":"LaGrange, IN"
}

In [16]:
#Replacing the county names
for key in combined_counties.keys():
    all_data_ei['GeoName'] = all_data_ei['GeoName'].replace(key, combined_counties[key])

for key in independent_cities.keys():
    all_data_ei['GeoName'] = all_data_ei['GeoName'].replace(key, independent_cities[key])
    
for key in remove_asterisk.keys():
    all_data_ei['GeoName'] = all_data_ei['GeoName'].replace(key, remove_asterisk[key])

In [17]:
#all_data_ei[all_data_ei.GeoName.str.contains("Maui")]

In [18]:
#all_data_ei[all_data_ei.GeoName.str.contains("Independent City")]
#all_data_ei[all_data_ei.GeoName.str.contains("+", regex=False)]
#all_data_ei[all_data_ei.GeoName.str.contains("*", regex=False)]

In [19]:
population_est = pd.read_csv("../co-est2019-alldata.csv", encoding = "ISO-8859-1")
population_est = population_est[['STNAME','CTYNAME','POPESTIMATE2010', 'POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019']]
population_est.head()

,STNAME,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,Alabama,Autauga County,54773,55227,54954,54727,54893,54864,55243,55390,55533,55869
2,Alabama,Baldwin County,183112,186558,190145,194885,199183,202939,207601,212521,217855,223234
3,Alabama,Barbour County,27327,27341,27169,26937,26755,26283,25806,25157,24872,24686
4,Alabama,Bibb County,22870,22745,22667,22521,22553,22566,22586,22550,22367,22394


In [20]:
population_est[population_est['STNAME'].str.contains("District")]

,STNAME,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
327,District of Columbia,District of Columbia,605226,619800,634924,650581,662328,675400,685815,694906,701547,705749
328,District of Columbia,District of Columbia,605226,619800,634924,650581,662328,675400,685815,694906,701547,705749


In [21]:
abcde = pd.read_csv('../co-est00int-tot.csv', encoding = "ISO-8859-1")
population_est['POPESTIMATE2009'] = abcde['POPESTIMATE2009'] #add 2019 estimates
population_est.columns = population_est.columns.str.replace("POPESTIMATE", "") #remove suffix
population_est.head()

,STNAME,CTYNAME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2009
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,4757938
1,Alabama,Autauga County,54773,55227,54954,54727,54893,54864,55243,55390,55533,55869,54135
2,Alabama,Baldwin County,183112,186558,190145,194885,199183,202939,207601,212521,217855,223234,179406
3,Alabama,Barbour County,27327,27341,27169,26937,26755,26283,25806,25157,24872,24686,27657
4,Alabama,Bibb County,22870,22745,22667,22521,22553,22566,22586,22550,22367,22394,22941


In [22]:
#remove states from the table
states = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
population_est = population_est[population_est.CTYNAME.isin(states) == False]

In [23]:
population_est = population_est.melt(id_vars=["STNAME", "CTYNAME"],
                                     var_name="Year",
                                     value_name = "Population Estimate")
#population_est

In [24]:
#Removing Parish and County
population_est['CTYNAME'] = population_est['CTYNAME'].str.replace(' Parish', '')
population_est['CTYNAME'] = population_est['CTYNAME'].str.replace(' County', '')

In [25]:
#Replacing state name with state abbreviation
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

population_est['STNAME'] = population_est['STNAME'].map(us_state_to_abbrev)
population_est

,STNAME,CTYNAME,Year,Population Estimate
0,AL,Autauga,2010,54773
1,AL,Baldwin,2010,183112
2,AL,Barbour,2010,27327
3,AL,Bibb,2010,22870
4,AL,Blount,2010,57376
...,...,...,...,...
34568,WY,Sweetwater,2009,10134
34569,WY,Teton,2009,44133
34570,WY,Uinta,2009,21232
34571,WY,Washakie,2009,21054


In [26]:
population_est["GeoName"] = population_est["CTYNAME"] + ", " + population_est["STNAME"]
#population_est[population_est.GeoName.str.contains("Baltimore")]

Combining all the counties and independent cities as done by the BEA:
- First check county/city population estimates to reference for later to see the number add up
- Add values to county using an array of years for querying
- Check county population again to see it add up
- There's 24 of these so this part is long
- This is very WET, open to better ideas but part of why I'm doing it this way is to be sure the numbers are adding up

In [27]:
population_est[population_est.GeoName.str.contains("Charlottesville city")]

,STNAME,CTYNAME,Year,Population Estimate,GeoName
2919,VA,Charlottesville city,2010,43461,"Charlottesville city, VA"
6062,VA,Charlottesville city,2011,43761,"Charlottesville city, VA"
9205,VA,Charlottesville city,2012,44580,"Charlottesville city, VA"
12348,VA,Charlottesville city,2013,45012,"Charlottesville city, VA"
15491,VA,Charlottesville city,2014,45518,"Charlottesville city, VA"
18634,VA,Charlottesville city,2015,46301,"Charlottesville city, VA"
21777,VA,Charlottesville city,2016,46943,"Charlottesville city, VA"
24920,VA,Charlottesville city,2017,47463,"Charlottesville city, VA"
28063,VA,Charlottesville city,2018,47506,"Charlottesville city, VA"
31206,VA,Charlottesville city,2019,47266,"Charlottesville city, VA"


In [28]:
#merged_df2

In [29]:
population_est[population_est["CTYNAME"] == "Charlottesville city"]

,STNAME,CTYNAME,Year,Population Estimate,GeoName
2919,VA,Charlottesville city,2010,43461,"Charlottesville city, VA"
6062,VA,Charlottesville city,2011,43761,"Charlottesville city, VA"
9205,VA,Charlottesville city,2012,44580,"Charlottesville city, VA"
12348,VA,Charlottesville city,2013,45012,"Charlottesville city, VA"
15491,VA,Charlottesville city,2014,45518,"Charlottesville city, VA"
18634,VA,Charlottesville city,2015,46301,"Charlottesville city, VA"
21777,VA,Charlottesville city,2016,46943,"Charlottesville city, VA"
24920,VA,Charlottesville city,2017,47463,"Charlottesville city, VA"
28063,VA,Charlottesville city,2018,47506,"Charlottesville city, VA"
31206,VA,Charlottesville city,2019,47266,"Charlottesville city, VA"


In [30]:
population_est.loc[population_est["CTYNAME"] == "Albemarle"]

,STNAME,CTYNAME,Year,Population Estimate,GeoName
2822,VA,Albemarle,2010,99204,"Albemarle, VA"
5965,VA,Albemarle,2011,100213,"Albemarle, VA"
9108,VA,Albemarle,2012,101464,"Albemarle, VA"
12251,VA,Albemarle,2013,102270,"Albemarle, VA"
15394,VA,Albemarle,2014,103732,"Albemarle, VA"
18537,VA,Albemarle,2015,105117,"Albemarle, VA"
21680,VA,Albemarle,2016,106431,"Albemarle, VA"
24823,VA,Albemarle,2017,107768,"Albemarle, VA"
27966,VA,Albemarle,2018,108377,"Albemarle, VA"
31109,VA,Albemarle,2019,109330,"Albemarle, VA"


In [31]:
#population_est.loc[2917, ["Population Estimate"]]["Population Estimate"]
population_est.loc[population_est[(population_est["Year"] == year) & (population_est["CTYNAME"] == "Charlottesville city")].index[0], "Population Estimate"]

47266

In [32]:
years = ["2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]

combined_two = [
    ['Maui, HI', 'Kalawao, HI'],
    ['Albemarle, VA', 'Charlottesville city, VA'],
    ['Alleghany, VA', 'Covington city, VA'],
    ['Campbell, VA', 'Lynchburg city, VA'],
    ['Carroll, VA', 'Galax city, VA'],
    ['Frederick, VA', 'Winchester city, VA'],
    ['Greensville, VA', 'Emporia city, VA'],
    ['Henry, VA', 'Martinsville city, VA'],
    ['James City, VA', 'Williamsburg city, VA'],
    ['Montgomery, VA', "Radford city, VA"],
    ['Pittsylvania, VA', 'Danville city, VA'],
    ['Prince George, VA', 'Hopewell city, VA'],
    ['Roanoke, VA', 'Salem city, VA'],
    ['Rockingham, VA', 'Harrisonburg city, VA'],
    ['Southampton, VA', 'Franklin city, VA'],
    ['Spotsylvania, VA', 'Fredericksburg city, VA'],
    ['Washington, VA', 'Bristol city, VA'],
    ['Wise, VA', 'Norton city, VA'],
    ['York, VA', 'Poquoson city, VA']
]

combined_three = [
    ['Augusta, VA', 'Staunton city, VA', 'Waynesboro city, VA'],
    ['Fairfax, VA', 'Fairfax city, VA', 'Falls Church city'],
    ['Rockbridge, VA', 'Buena Vista city, VA', 'Lezington city, VA'],
    ['Dinwiddie, VA', 'Colonial Heights city, VA', 'Petersburg city, VA'],
    ['Prince William, VA', 'Manassas city, VA', 'Manassas Park city, VA']
]

In [33]:
for x in range(len(combined_two)):
    for year in years:
        population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_two[x][0])].index[0], "Population Estimate"] = population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_two[x][1])].index[0], "Population Estimate"] + population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_two[x][0])].index[0], "Population Estimate"]

In [34]:
for x in range(len(combined_three)):
    for year in years:
        population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_three[x][0])].index[0], "Population Estimate"] = population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_three[x][1])].index[0], "Population Estimate"] + population_est.loc[population_est[(population_est["Year"] == year) & (population_est["GeoName"] == combined_three[x][0])].index[0], "Population Estimate"]

In [35]:
#Checking
#population_est[population_est["CTYNAME"] == "Albemarle"]
#population_est[population_est.GeoName.str.contains("Fairfax, VA")]

In [36]:
population_est

,STNAME,CTYNAME,Year,Population Estimate,GeoName
0,AL,Autauga,2010,54773,"Autauga, AL"
1,AL,Baldwin,2010,183112,"Baldwin, AL"
2,AL,Barbour,2010,27327,"Barbour, AL"
3,AL,Bibb,2010,22870,"Bibb, AL"
4,AL,Blount,2010,57376,"Blount, AL"
...,...,...,...,...,...
34568,WY,Sweetwater,2009,10134,"Sweetwater, WY"
34569,WY,Teton,2009,44133,"Teton, WY"
34570,WY,Uinta,2009,21232,"Uinta, WY"
34571,WY,Washakie,2009,21054,"Washakie, WY"


In [37]:
population_est.rename(columns = {'Year':'year'}, inplace = True)

In [38]:
population_est["GeoName"] = population_est["GeoName"].str.replace(r'city', 'City')

In [39]:
merger_df = pd.merge(all_data_ei, population_est, on=["GeoName", "year"], how="left")
#merger_df[merger_df.GeoFIPS.isnull()] checking nulls

# Adding Business Applications

In [40]:
cdbg_funding = pd.read_excel("../Workbook2.xlsx", skiprows=3)
cdbg_funding.head()

,Year,State,CoC Name,Org Name,Program Name,Program Type,Award Amount
0,2021,AK,NaN,Alaska,CDBG,NaN,3348442.0
1,2021,AK,NaN,"Anchorage, AK",CDBG,NaN,1948478.0
2,2021,AL,NaN,Alabama,CDBG,NaN,24256102.0
3,2021,AL,NaN,"Anniston, AL",CDBG,NaN,563039.0
4,2021,AL,NaN,"Auburn, AL",CDBG,NaN,686092.0


In [41]:
#checking Baltimore for county vs city
#cdbg_funding[cdbg_funding["Org Name"].str.contains("Baltimore")]

In [42]:
#for easier referencing
cdbg_funding.columns = [c.replace(' ', '_') for c in cdbg_funding.columns]
#removing states
cdbg_funding = cdbg_funding[cdbg_funding.Org_Name.isin(states) == False]
cdbg_funding.head()

,Year,State,CoC_Name,Org_Name,Program_Name,Program_Type,Award_Amount
1,2021,AK,NaN,"Anchorage, AK",CDBG,NaN,1948478.0
3,2021,AL,NaN,"Anniston, AL",CDBG,NaN,563039.0
4,2021,AL,NaN,"Auburn, AL",CDBG,NaN,686092.0
5,2021,AL,NaN,"Bessemer, AL",CDBG,NaN,598741.0
6,2021,AL,NaN,"Birmingham, AL",CDBG,NaN,6054064.0


In [43]:
#Checking States
#cdbg_funding.loc[cdbg_funding["State"] == "NJ"].head()
#cdbg_funding.loc[cdbg_funding["State"] == "AK"].head(30)

In [44]:
bfs = pd.read_excel("../bfs_county_apps_annual.xlsx", skiprows=2)
bfs.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,...,BA2012,BA2013,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021
0,AL,Autauga County,1001,1,1,335,348,348,367,288,...,242,282,290,291,294,313,384,394,548,763
1,AL,Baldwin County,1003,1,3,1986,2101,1975,1721,1505,...,1594,1532,1575,1735,1858,2114,2104,2317,2686,3385
2,AL,Barbour County,1005,1,5,138,179,156,139,133,...,120,121,120,129,122,114,113,135,236,331
3,AL,Bibb County,1007,1,7,118,145,117,98,98,...,95,83,101,78,86,79,105,86,160,190
4,AL,Blount County,1009,1,9,315,323,297,306,247,...,228,225,228,218,254,281,299,312,344,462


In [45]:
#bfs[bfs.State == "GA"]

In [46]:
bfs = bfs.drop(['BA2005','BA2006','BA2007','BA2008', 'BA2020', 'BA2021'], axis=1)
bfs.head()

,State,County,County Code,fipstate,fipscty,BA2009,BA2010,BA2011,BA2012,BA2013,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019
0,AL,Autauga County,1001,1,1,288,304,278,242,282,290,291,294,313,384,394
1,AL,Baldwin County,1003,1,3,1505,1548,1553,1594,1532,1575,1735,1858,2114,2104,2317
2,AL,Barbour County,1005,1,5,133,107,101,120,121,120,129,122,114,113,135
3,AL,Bibb County,1007,1,7,98,106,72,95,83,101,78,86,79,105,86
4,AL,Blount County,1009,1,9,247,247,263,228,225,228,218,254,281,299,312


In [47]:
bfs.columns = bfs.columns.str.lstrip("BA")
bfs.head()

,State,County,County Code,fipstate,fipscty,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AL,Autauga County,1001,1,1,288,304,278,242,282,290,291,294,313,384,394
1,AL,Baldwin County,1003,1,3,1505,1548,1553,1594,1532,1575,1735,1858,2114,2104,2317
2,AL,Barbour County,1005,1,5,133,107,101,120,121,120,129,122,114,113,135
3,AL,Bibb County,1007,1,7,98,106,72,95,83,101,78,86,79,105,86
4,AL,Blount County,1009,1,9,247,247,263,228,225,228,218,254,281,299,312


In [48]:
bfs = bfs.melt(id_vars=["State", "County", "County Code", "fipstate", "fipscty"],
        var_name="Year",
        value_name = "Applications")

In [49]:
bfs

,State,County,County Code,fipstate,fipscty,Year,Applications
0,AL,Autauga County,1001,1,1,2009,288
1,AL,Baldwin County,1003,1,3,2009,1505
2,AL,Barbour County,1005,1,5,2009,133
3,AL,Bibb County,1007,1,7,2009,98
4,AL,Blount County,1009,1,9,2009,247
...,...,...,...,...,...,...,...
34612,WY,Sweetwater County,56037,56,37,2019,416
34613,WY,Teton County,56039,56,39,2019,1817
34614,WY,Uinta County,56041,56,41,2019,190
34615,WY,Washakie County,56043,56,43,2019,96


In [50]:
bfs["GeoName"] = bfs["County"] + ", " + bfs["State"]
bfs.head()

,State,County,County Code,fipstate,fipscty,Year,Applications,GeoName
0,AL,Autauga County,1001,1,1,2009,288,"Autauga County, AL"
1,AL,Baldwin County,1003,1,3,2009,1505,"Baldwin County, AL"
2,AL,Barbour County,1005,1,5,2009,133,"Barbour County, AL"
3,AL,Bibb County,1007,1,7,2009,98,"Bibb County, AL"
4,AL,Blount County,1009,1,9,2009,247,"Blount County, AL"


In [51]:
bfs.rename(columns = {'Year':'year'}, inplace = True)

In [52]:
#I think this section was from before I just didn't extract those years
#all_data_ei.year = all_data_ei.year.astype(int)

In [53]:
#all_data_ei = all_data_ei.drop(all_data_ei[all_data_ei.year < 2005].index)
#all_data_ei

In [54]:
#all_data_ei = all_data_ei.sort_values(["GeoName", "year"])
#bfs = bfs.sort_values(["GeoName", "year"])

In [55]:
#all_data_ei[all_data_ei.GeoName.str.contains("Baltimore")]
#bfs['GeoName'] = bfs['GeoName'].str.replace(r'AK', 'AK\*')
#bfs[bfs.State == "AK"]
#bfs[bfs['GeoName'].str.contains("Weston, WY")]

In [56]:
bfs[bfs.GeoName.str.contains("Albemarle")]

,State,County,County Code,fipstate,fipscty,year,Applications,GeoName
2825,VA,Albemarle County,51003,51,3,2009,680,"Albemarle County, VA"
5972,VA,Albemarle County,51003,51,3,2010,726,"Albemarle County, VA"
9119,VA,Albemarle County,51003,51,3,2011,753,"Albemarle County, VA"
12266,VA,Albemarle County,51003,51,3,2012,811,"Albemarle County, VA"
15413,VA,Albemarle County,51003,51,3,2013,857,"Albemarle County, VA"
18560,VA,Albemarle County,51003,51,3,2014,785,"Albemarle County, VA"
21707,VA,Albemarle County,51003,51,3,2015,843,"Albemarle County, VA"
24854,VA,Albemarle County,51003,51,3,2016,854,"Albemarle County, VA"
28001,VA,Albemarle County,51003,51,3,2017,958,"Albemarle County, VA"
31148,VA,Albemarle County,51003,51,3,2018,960,"Albemarle County, VA"


In [57]:
bfs[bfs.GeoName.str.contains("Fairfax")]

,State,County,County Code,fipstate,fipscty,year,Applications,GeoName
2852,VA,Fairfax County,51059,51,59,2009,11185,"Fairfax County, VA"
2929,VA,Fairfax city,51600,51,600,2009,437,"Fairfax city, VA"
5999,VA,Fairfax County,51059,51,59,2010,11130,"Fairfax County, VA"
6076,VA,Fairfax city,51600,51,600,2010,440,"Fairfax city, VA"
9146,VA,Fairfax County,51059,51,59,2011,12037,"Fairfax County, VA"
9223,VA,Fairfax city,51600,51,600,2011,474,"Fairfax city, VA"
12293,VA,Fairfax County,51059,51,59,2012,12402,"Fairfax County, VA"
12370,VA,Fairfax city,51600,51,600,2012,507,"Fairfax city, VA"
15440,VA,Fairfax County,51059,51,59,2013,12660,"Fairfax County, VA"
15517,VA,Fairfax city,51600,51,600,2013,501,"Fairfax city, VA"


In [58]:
#Remove county from GeoName
bfs["GeoName"] = bfs["GeoName"].str.replace(r' County', '')
bfs["GeoName"] = bfs["GeoName"].str.replace(r' Parish', '')

In [59]:
bfs['Applications'] = bfs['Applications'].str.replace(r' ', '0')
#bfs[bfs['Applications'].str.contains(' ')]

In [60]:
#Turning Applications from a string to int
bfs['Applications'] = bfs['Applications'].astype(int)

In [61]:
for x in range(len(combined_two)):
    for year in years:
        bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_two[x][0])].index[0], "Applications"] = bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_two[x][1])].index[0], "Applications"] + bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_two[x][0])].index[0], "Applications"]

In [62]:
bfs[bfs.GeoName.str.contains("Albemarle")]

,State,County,County Code,fipstate,fipscty,year,Applications,GeoName
2825,VA,Albemarle County,51003,51,3,2009,1133,"Albemarle, VA"
5972,VA,Albemarle County,51003,51,3,2010,1178,"Albemarle, VA"
9119,VA,Albemarle County,51003,51,3,2011,1258,"Albemarle, VA"
12266,VA,Albemarle County,51003,51,3,2012,1315,"Albemarle, VA"
15413,VA,Albemarle County,51003,51,3,2013,1394,"Albemarle, VA"
18560,VA,Albemarle County,51003,51,3,2014,1336,"Albemarle, VA"
21707,VA,Albemarle County,51003,51,3,2015,1381,"Albemarle, VA"
24854,VA,Albemarle County,51003,51,3,2016,1399,"Albemarle, VA"
28001,VA,Albemarle County,51003,51,3,2017,1581,"Albemarle, VA"
31148,VA,Albemarle County,51003,51,3,2018,1554,"Albemarle, VA"


In [63]:
for x in range(len(combined_three)):
    for year in years:
        bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_three[x][0])].index[0], "Applications"] = bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_three[x][1])].index[0], "Applications"] + bfs.loc[bfs[(bfs["year"] == year) & (bfs["GeoName"] == combined_three[x][0])].index[0], "Applications"]

In [64]:
bfs[bfs.GeoName.str.contains("Fairfax")]

,State,County,County Code,fipstate,fipscty,year,Applications,GeoName
2852,VA,Fairfax County,51059,51,59,2009,11622,"Fairfax, VA"
2929,VA,Fairfax city,51600,51,600,2009,437,"Fairfax city, VA"
5999,VA,Fairfax County,51059,51,59,2010,11570,"Fairfax, VA"
6076,VA,Fairfax city,51600,51,600,2010,440,"Fairfax city, VA"
9146,VA,Fairfax County,51059,51,59,2011,12511,"Fairfax, VA"
9223,VA,Fairfax city,51600,51,600,2011,474,"Fairfax city, VA"
12293,VA,Fairfax County,51059,51,59,2012,12909,"Fairfax, VA"
12370,VA,Fairfax city,51600,51,600,2012,507,"Fairfax city, VA"
15440,VA,Fairfax County,51059,51,59,2013,13161,"Fairfax, VA"
15517,VA,Fairfax city,51600,51,600,2013,501,"Fairfax city, VA"


In [65]:
bfs["GeoName"] = bfs["GeoName"].str.replace(r'city', 'City')

In [66]:
#change La Salle to LaSalle
#fix Doña Ana
#bfs[bfs["County Code"] == 35013]

fixes_bfs = {
    'Doâ”a Ana, NM': 'Doña Ana, NM', 
    'La Salle, LA':'LaSalle, LA',
}

In [67]:
for key in fixes_bfs.keys():
    bfs['GeoName'] = bfs['GeoName'].replace(key, fixes_bfs[key])

In [68]:
merged_df2 = pd.merge(merger_df, bfs, on=["GeoName", "year"], how="left")
merged_df2

,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,year,STNAME,CTYNAME,Population Estimate,State,County,County Code,fipstate,fipscty,Applications
0,"""01001""","Autauga, AL",535644,39092,69250,28256,1160479,11424,8731,67348,...,2009,AL,Autauga,54135,AL,Autauga County,1001,1,1,288
1,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,2009,AL,Baldwin,179406,AL,Baldwin County,1003,1,3,1505
2,"""01005""","Barbour, AL",429668,28956,64008,23435,387341,18712,15908,52391,...,2009,AL,Barbour,27657,AL,Barbour County,1005,1,5,133
3,"""01007""","Bibb, AL",206853,15973,29693,10729,321855,2655,2515,26702,...,2009,AL,Bibb,22941,AL,Bibb County,1007,1,7,98
4,"""01009""","Blount, AL",462562,36182,56706,22040,993044,20257,16640,58222,...,2009,AL,Blount,57341,AL,Blount County,1009,1,9,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34249,"""56037""","Sweetwater, WY",2186822,122914,243513,125358,1778004,9846,8485,248272,...,2019,WY,Sweetwater,42343,WY,Sweetwater County,56037,56,37,416
34250,"""56039""","Teton, WY",1858314,102201,137206,107871,1519128,3364,-483,210072,...,2019,WY,Teton,23464,WY,Teton County,56039,56,39,1817
34251,"""56041""","Uinta, WY",546044,34219,79408,36393,536355,4720,2823,70612,...,2019,WY,Uinta,20226,WY,Uinta County,56041,56,41,190
34252,"""56043""","Washakie, WY",255080,15662,33784,15405,218093,8598,4675,31067,...,2019,WY,Washakie,7805,WY,Washakie County,56043,56,43,96


# Add CDBG funding

In [69]:
cdbg_funding = pd.read_excel("../Workbook2.xlsx", skiprows=3)
cdbg_funding.head()

,Year,State,CoC Name,Org Name,Program Name,Program Type,Award Amount
0,2021,AK,NaN,Alaska,CDBG,NaN,3348442.0
1,2021,AK,NaN,"Anchorage, AK",CDBG,NaN,1948478.0
2,2021,AL,NaN,Alabama,CDBG,NaN,24256102.0
3,2021,AL,NaN,"Anniston, AL",CDBG,NaN,563039.0
4,2021,AL,NaN,"Auburn, AL",CDBG,NaN,686092.0


In [70]:
#Getting rid of states
cdbg_funding = cdbg_funding[cdbg_funding["Org Name"].isin(states) == False]

In [71]:
#Getting rid of data from territories
territories = ["MP", "PR"]
cdbg_funding = cdbg_funding[cdbg_funding.State.isin(territories) == False]

In [72]:
cdbg_funding = cdbg_funding.drop(columns=["CoC Name", "Program Name", "Program Type"])

In [73]:
crosswalk = pd.read_csv("../location_crosswalk.csv")
#crosswalk[crosswalk.GEOID == 1073]

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/4028801675.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crosswalk = pd.read_csv("../location_crosswalk.csv")


In [74]:
_county_crosswalk = pd.read_csv("../crosswalk_county_name_to_county_fips.csv")
_county_crosswalk.head()

,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,31,39,31039,Cuming,Nebraska
1,53,69,53069,Wahkiakum,Washington
2,35,11,35011,De Baca,New Mexico
3,31,109,31109,Lancaster,Nebraska
4,31,129,31129,Nuckolls,Nebraska


In [75]:
crosswalk = crosswalk.drop(columns=["id", "country", "pv_lat", "gl_lat", "pv_long", "gl_long"])
crosswalk.head()

,GEOID,pv_city,gl_city,pv_state,gl_state,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,6075,San Francisco,NaN,CA,NaN,San Francisco,NaN,6.0,6075.0,NaN
1,32031,Reno,NaN,NV,NaN,Washoe,NaN,32.0,32031.0,NaN
2,50021,Rutland,NaN,VT,NaN,Rutland,NaN,50.0,50021.0,NaN
3,34003,Parkridge,Park Ridge,NJ,NJ,NaN,Bergen,34.0,NaN,34003.0
4,6085,Sunnyvale,NaN,CA,NaN,Santa Clara,NaN,6.0,6085.0,NaN


In [76]:
crosswalk.isna().sum()

GEOID                 0
pv_city               0
gl_city           18380
pv_state              0
gl_state          18380
pv_county         22572
gl_county         18380
pv_state_fips        39
pv_county_fips    22572
gl_county_fips    18380
dtype: int64

In [77]:
merged_crosswalk = pd.merge(crosswalk, _county_crosswalk, on=["GEOID"], how="inner")
merged_crosswalk.isna().sum()

GEOID                 0
pv_city               0
gl_city           16755
pv_state              0
gl_state          16755
pv_county         16010
gl_county         16755
pv_state_fips        28
pv_county_fips    16010
gl_county_fips    16755
STATEFP               0
COUNTYFP              0
county_name           0
state_name            0
dtype: int64

In [78]:
#county_list = merged_crosswalk.county_name.unique()
#county_list1 = merged_crosswalk.pv_county.unique()

In [79]:
#merged_df2.GeoName.unique()

In [80]:
merged_crosswalk[merged_crosswalk.pv_city.str.contains("Baltimore")]

,GEOID,pv_city,gl_city,pv_state,gl_state,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips,STATEFP,COUNTYFP,county_name,state_name
7484,24005,Baltimore County,8,MD,MD,NaN,Baltimore,24.0,NaN,24005.0,24,5,Baltimore,Maryland
7526,24005,Baltimore,8,PA,MD,NaN,Baltimore,42.0,NaN,24005.0,24,5,Baltimore,Maryland
9586,24510,Baltimore,NaN,MD,NaN,Baltimore (city),NaN,24.0,24510.0,NaN,24,510,Baltimore,Maryland
9599,24510,Baltimore,Baltimore,MS,MD,NaN,Baltimore,28.0,NaN,24510.0,24,510,Baltimore,Maryland
9604,24510,Baltimore,Baltimore,MA,MD,NaN,Baltimore,25.0,NaN,24510.0,24,510,Baltimore,Maryland
12136,26099,New Baltimore,NaN,MI,NaN,Macomb,NaN,26.0,26099.0,NaN,26,99,Macomb,Michigan
15713,6061,Baltimore,Foresthill-Back Country,CA,CA,NaN,Placer,6.0,NaN,6061.0,6,61,Placer,California
17177,39173,North Baltimore,NaN,OH,NaN,Wood,NaN,39.0,39173.0,NaN,39,173,Wood,Ohio
18620,50027,Baltimore,Baltimore,VT,VT,NaN,Windsor,50.0,NaN,50027.0,50,27,Windsor,Vermont
24356,39045,Baltimore,NaN,OH,NaN,Fairfield,NaN,39.0,39045.0,NaN,39,45,Fairfield,Ohio


In [81]:
cdbg_funding = cdbg_funding[cdbg_funding.Year < 2020]
cdbg_funding

,Year,State,Org Name,Award Amount
4989,2019,AK,"Anchorage, AK",1720154.0
4991,2019,AL,"Anniston, AL",556454.0
4992,2019,AL,"Auburn, AL",592783.0
4993,2019,AL,"Bessemer, AL",577199.0
4994,2019,AL,"Birmingham, AL",5870169.0
...,...,...,...,...
25596,2003,WV,"Parkersburg, WV",1286000.0
25597,2003,WV,"Weirton, WV",596000.0
25599,2003,WV,"Wheeling, WV",1820000.0
25600,2003,WY,"Casper, WY",551000.0


In [82]:
cdbg_funding = cdbg_funding[cdbg_funding.Year > 2008]
cdbg_funding

,Year,State,Org Name,Award Amount
4989,2019,AK,"Anchorage, AK",1720154.0
4991,2019,AL,"Anniston, AL",556454.0
4992,2019,AL,"Auburn, AL",592783.0
4993,2019,AL,"Bessemer, AL",577199.0
4994,2019,AL,"Birmingham, AL",5870169.0
...,...,...,...,...
18597,2009,WV,"Vienna, WV",113834.0
18598,2009,WV,"Weirton, WV",496810.0
18600,2009,WV,"Wheeling, WV",1478940.0
18601,2009,WY,"Casper, WY",440491.0


In [83]:
crosswalk.pv_county.fillna(crosswalk.gl_county, inplace=True)

In [84]:
crosswalk["County_Name"] = crosswalk["pv_county"] + ", " + crosswalk["pv_state"]
crosswalk.head()

,GEOID,pv_city,gl_city,pv_state,gl_state,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips,County_Name
0,6075,San Francisco,NaN,CA,NaN,San Francisco,NaN,6.0,6075.0,NaN,"San Francisco, CA"
1,32031,Reno,NaN,NV,NaN,Washoe,NaN,32.0,32031.0,NaN,"Washoe, NV"
2,50021,Rutland,NaN,VT,NaN,Rutland,NaN,50.0,50021.0,NaN,"Rutland, VT"
3,34003,Parkridge,Park Ridge,NJ,NJ,Bergen,Bergen,34.0,NaN,34003.0,"Bergen, NJ"
4,6085,Sunnyvale,NaN,CA,NaN,Santa Clara,NaN,6.0,6085.0,NaN,"Santa Clara, CA"


In [85]:
crosswalk["Org_Name"] = crosswalk["pv_city"] + ", " + crosswalk["pv_state"]
crosswalk.head()

,GEOID,pv_city,gl_city,pv_state,gl_state,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips,County_Name,Org_Name
0,6075,San Francisco,NaN,CA,NaN,San Francisco,NaN,6.0,6075.0,NaN,"San Francisco, CA","San Francisco, CA"
1,32031,Reno,NaN,NV,NaN,Washoe,NaN,32.0,32031.0,NaN,"Washoe, NV","Reno, NV"
2,50021,Rutland,NaN,VT,NaN,Rutland,NaN,50.0,50021.0,NaN,"Rutland, VT","Rutland, VT"
3,34003,Parkridge,Park Ridge,NJ,NJ,Bergen,Bergen,34.0,NaN,34003.0,"Bergen, NJ","Parkridge, NJ"
4,6085,Sunnyvale,NaN,CA,NaN,Santa Clara,NaN,6.0,6085.0,NaN,"Santa Clara, CA","Sunnyvale, CA"


In [86]:
crosswalk = crosswalk.drop(columns=["gl_city", "gl_state", "gl_county", "pv_state_fips", "pv_county_fips", "gl_county_fips"],axis = 1)
crosswalk = crosswalk.drop(columns=["pv_city", "pv_state", "pv_county"], axis = 1)
crosswalk.head()

,GEOID,County_Name,Org_Name
0,6075,"San Francisco, CA","San Francisco, CA"
1,32031,"Washoe, NV","Reno, NV"
2,50021,"Rutland, VT","Rutland, VT"
3,34003,"Bergen, NJ","Parkridge, NJ"
4,6085,"Santa Clara, CA","Sunnyvale, CA"


In [87]:
#Remove county and Parish
#cdbg_funding.loc['Org Name'] = cdbg_funding['Org Name'].str.replace(r' County', '')
#cdbg_funding.loc['Org Name'] = cdbg_funding['Org Name'].str.replace(r' Parish', '')

In [88]:
cdbg_funding.rename(columns={'Org Name':'Org_Name'}, inplace = True)

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/883871368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdbg_funding.rename(columns={'Org Name':'Org_Name'}, inplace = True)


In [89]:
county_list = merged_df2.GeoName.unique()

In [90]:
county_list

array(['Autauga, AL', 'Baldwin, AL', 'Barbour, AL', ..., 'Uinta, WY',
       'Washakie, WY', 'Weston, WY'], dtype=object)

In [91]:
crosswalk.shape

(40952, 3)

In [92]:
crosswalk

,GEOID,County_Name,Org_Name
0,6075,"San Francisco, CA","San Francisco, CA"
1,32031,"Washoe, NV","Reno, NV"
2,50021,"Rutland, VT","Rutland, VT"
3,34003,"Bergen, NJ","Parkridge, NJ"
4,6085,"Santa Clara, CA","Sunnyvale, CA"
...,...,...,...
40947,18011,"Boone, IN","Zlonsville, IN"
40948,29031,"Cape Girardeau, CA","Santa C, CA"
40949,06013,"Contra Costa, CA","Kinsington, CA"
40950,42111,"Somerset, PA","Tire Hill, PA"


In [93]:
county_geoid = _county_crosswalk.copy()
county_geoid.head()

,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,31,39,31039,Cuming,Nebraska
1,53,69,53069,Wahkiakum,Washington
2,35,11,35011,De Baca,New Mexico
3,31,109,31109,Lancaster,Nebraska
4,31,129,31129,Nuckolls,Nebraska


In [94]:
county_geoid["County_Name"] = county_geoid["county_name"] + ", " + county_geoid["state_name"]
county_geoid.head()

,STATEFP,COUNTYFP,GEOID,county_name,state_name,County_Name
0,31,39,31039,Cuming,Nebraska,"Cuming, Nebraska"
1,53,69,53069,Wahkiakum,Washington,"Wahkiakum, Washington"
2,35,11,35011,De Baca,New Mexico,"De Baca, New Mexico"
3,31,109,31109,Lancaster,Nebraska,"Lancaster, Nebraska"
4,31,129,31129,Nuckolls,Nebraska,"Nuckolls, Nebraska"


In [95]:
#merge_geoid = pd.merge(county_geoid, county_list, on=["County_Name"], how="inner")
#merge_geoid.head()

In [96]:
for x in range(len(county_list)):
    crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
crosswalk.shape

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

...

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

(44065, 3)

In [97]:
no_geoid = crosswalk[crosswalk.GEOID.isnull()]
#no_geoid.shape
no_geoid = no_geoid.drop(columns=["GEOID"])
#_county_crosswalk.shape

In [98]:
county_crosswalk = _county_crosswalk.copy()
county_crosswalk['state_name'] = county_crosswalk['state_name'].map(us_state_to_abbrev)
county_crosswalk['Org_Name'] = county_crosswalk['county_name'] + ", " + county_crosswalk['state_name']
county_crosswalk

,STATEFP,COUNTYFP,GEOID,county_name,state_name,Org_Name
0,31,39,31039,Cuming,NE,"Cuming, NE"
1,53,69,53069,Wahkiakum,WA,"Wahkiakum, WA"
2,35,11,35011,De Baca,NM,"De Baca, NM"
3,31,109,31109,Lancaster,NE,"Lancaster, NE"
4,31,129,31129,Nuckolls,NE,"Nuckolls, NE"
...,...,...,...,...,...,...
3229,13,123,13123,Gilmer,GA,"Gilmer, GA"
3230,27,135,27135,Roseau,MN,"Roseau, MN"
3231,28,89,28089,Madison,MS,"Madison, MS"
3232,48,227,48227,Howard,TX,"Howard, TX"


In [99]:
fixed_geoid = pd.merge(no_geoid, county_crosswalk, how='left', on=["Org_Name"])

In [100]:
##fixed_geoid[fixed_geoid.GEOID.isnull()]

In [101]:
##More work
fixed_geoid.loc[fixed_geoid.County_Name == "Aleutians East Borough, AK", 'GEOID'] = 2013
fixed_geoid.loc[fixed_geoid.County_Name == "Aleutians West Census Area, AK", 'GEOID'] = 2016
fixed_geoid.loc[fixed_geoid.County_Name == "Anchorage Municipality, AK", 'GEOID'] = 2020
fixed_geoid.loc[fixed_geoid.County_Name == "Bethel Census Area, AK", 'GEOID'] = 2050
fixed_geoid.loc[fixed_geoid.County_Name == "Bristol Bay Borough, AK", 'GEOID'] = 2060
fixed_geoid.loc[fixed_geoid.County_Name == "Denali Borough, AK", 'GEOID'] = 2068
fixed_geoid.loc[fixed_geoid.County_Name == "Dillingham Census Area, AK", 'GEOID'] = 2070
fixed_geoid.loc[fixed_geoid.County_Name == "Fairbanks North Star Borough, AK", 'GEOID'] = 2090
fixed_geoid.loc[fixed_geoid.County_Name == "Haines Borough, AK", 'GEOID'] = 2100
fixed_geoid.loc[fixed_geoid.County_Name == "Hoonah-Angoon Census Area, AK", 'GEOID'] = 2105
fixed_geoid.loc[fixed_geoid.County_Name == "Juneau City and Borough, AK", 'GEOID'] = 2110
fixed_geoid.loc[fixed_geoid.County_Name == "Kenai Peninsula Borough, AK", 'GEOID'] = 2122
fixed_geoid.loc[fixed_geoid.County_Name == "Ketchikan Gateway Borough, AK", 'GEOID'] = 2130
fixed_geoid.loc[fixed_geoid.County_Name == "Kodiak Island Borough, AK", 'GEOID'] = 2150
fixed_geoid.loc[fixed_geoid.County_Name == "Kusilvak Census Area, AK", 'GEOID'] = 2158
fixed_geoid.loc[fixed_geoid.County_Name == "Lake and Peninsula Borough, AK", 'GEOID'] = 2164
fixed_geoid.loc[fixed_geoid.County_Name == "Matanuska-Susitna Borough, AK", 'GEOID'] = 2170
fixed_geoid.loc[fixed_geoid.County_Name == "Nome Census Area, AK", 'GEOID'] = 2180
fixed_geoid.loc[fixed_geoid.County_Name == "North Slope Borough, AK", 'GEOID'] = 2185
fixed_geoid.loc[fixed_geoid.County_Name == "Northwest Arctic Borough, AK", 'GEOID'] = 2188
fixed_geoid.loc[fixed_geoid.County_Name == "Petersburg Borough, AK", 'GEOID'] = 2195

In [102]:
crosswalk = crosswalk[crosswalk.GEOID.isnull() == False]

In [103]:
fixed_crosswalk = pd.concat([crosswalk, fixed_geoid])

In [104]:
fixed_crosswalk[fixed_crosswalk.GEOID.isnull()].shape

(20, 7)

In [105]:
for x in range(len(county_list)):
    crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
crosswalk.shape

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

...

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2214757123.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crosswalk = crosswalk.append({'County_Name': county_list[x],'Org_Name': county_list[x]}, ignore_index = True)
/var/folders/v1/7rn

(44065, 3)

In [106]:
#Remove county and Parish
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' County', '')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' Parish', '')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' Township', '')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' Village', '')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'San Buenaventura', 'Ventura')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Dekalb', 'DeKalb')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Fond Du Lac', 'Fond du Lac')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Mckinney', 'McKinney')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Mc Allen', 'McAllen')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Saint Johns, FL', 'St. Johns, FL')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Brookhaven Town, NY', 'Brookhaven, NY')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Lower Merion, PA', 'Lower Merion Township, PA')
crosswalk["Org_Name"] = crosswalk["Org_Name"].str.replace(r'Union City, NY', 'Union, NY')
crosswalk["Org_Name"] = crosswalk["Org_Name"].str.replace(r'O Fallon, MO', "O'Fallon, MO")
crosswalk["Org_Name"] = crosswalk["Org_Name"].str.replace(r'O.Fallon, MO', "O'Fallon, MO")
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Desoto, TX', 'DeSoto, TX')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'West Valley, UT', 'West Valley City, UT')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Mckeesport, PA', 'McKeesport, PA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Nashville-Davidson, TN', 'Nashville, TN')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Jacksonville-Duval, FL', 'Jacksonville, FL')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Athens-Clarke, GA', 'Athens, GA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Augusta-Richmond, GA', 'Augusta, GA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Columbus-Muscogee, GA', 'Columbus, GA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Macon-Bibb, GA', 'Macon, GA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Lexington-Fayette, KY', 'Lexington, KY')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Louisville-Jefferson, KY', 'Louisville, KY')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Houma-Terrebonne, LA', 'Houma, LA')
cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r'Parsippany-Troyhills, NJ', 'Parsippany-Troy Hills, NJ')

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/1879561299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' County', '')
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/1879561299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdbg_funding['Org_Name'] = cdbg_funding['Org_Name'].str.replace(r' Parish', '')
/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/1879561299.py:4: SettingWithCop

In [107]:
#crosswalk[crosswalk["County_Name"].str.contains("Salt Lake")].head(30)

In [108]:
cdbg_funding[cdbg_funding["Org_Name"].str.contains("Clarke")]

,Year,State,Org_Name,Award Amount


In [109]:
merged_df3 = cdbg_funding.merge(crosswalk, on=["Org_Name"], how='left')
#Crosswalk does not have Palm Coast, FL
#or Coeur d'Alene, ID
merged_df3[(merged_df3.County_Name.isnull())]

,Year,State,Org_Name,Award Amount,GEOID,County_Name
395,2019,FL,"Palm Coast, FL",487895.0,NaN,NaN
481,2019,ID,"Coeur d'Alene, ID",329815.0,NaN,NaN
1787,2018,FL,"Palm Coast, FL",506673.0,NaN,NaN
1872,2018,ID,"Coeur d'Alene, ID",318476.0,NaN,NaN
3184,2017,FL,"Palm Coast, FL",485286.0,NaN,NaN
3270,2017,ID,"Coeur d'Alene, ID",301850.0,NaN,NaN
4613,2016,FL,"Palm Coast, FL",474932.0,NaN,NaN
4700,2016,ID,"Coeur d'Alene, ID",310681.0,NaN,NaN
6045,2015,FL,"Palm Coast, FL",438979.0,NaN,NaN
6129,2015,ID,"Coeur d'Alene, ID",291050.0,NaN,NaN


In [110]:
merged_df3['County_Name'].mask(merged_df3['Org_Name'] == 'Palm Coast, FL', "Flagler, FL", inplace=True)
merged_df3['County_Name'].mask(merged_df3['Org_Name'] == "Coeur d'Alene, ID", "Kootenai, ID", inplace=True)
merged_df3[(merged_df3.GEOID.isnull())].head(20)

,Year,State,Org_Name,Award Amount,GEOID,County_Name
11,2019,AL,"Jefferson, AL",2330492.0,NaN,"Jefferson, AL"
13,2019,AL,"Mobile, AL",1518243.0,NaN,"Mobile, AL"
15,2019,AL,"Mobile, AL",2375186.0,NaN,"Mobile, AL"
17,2019,AL,"Montgomery, AL",1665597.0,NaN,"Montgomery, AL"
20,2019,AL,"Tuscaloosa, AL",783595.0,NaN,"Tuscaloosa, AL"
22,2019,AR,"Conway, AR",472390.0,NaN,"Conway, AR"
43,2019,AZ,"Maricopa, AZ",2989846.0,NaN,"Maricopa, AZ"
49,2019,AZ,"Pima, AZ",2945544.0,NaN,"Pima, AZ"
50,2019,AZ,"Pinal, AZ",1301489.0,NaN,"Pinal, AZ"
58,2019,AZ,"Yuma, AZ",843078.0,NaN,"Yuma, AZ"


In [111]:
#cdbg_funding.tail()
#cdbg_funding.drop(cdbg_funding.tail(1).index,inplace=True)
cdbg_funding[cdbg_funding["Org_Name"].str.contains("Jefferson")]

,Year,State,Org_Name,Award Amount
5001,2019,AL,"Jefferson, AL",2330492.0
5241,2019,CO,"Jefferson, CO",1036293.0
5501,2019,LA,"Jefferson, LA",2495382.0
5639,2019,MO,"Jefferson City, MO",292733.0
5640,2019,MO,"Jefferson, MO",1175850.0
6230,2018,AL,"Jefferson, AL",2355502.0
6469,2018,CO,"Jefferson, CO",1052175.0
6727,2018,LA,"Jefferson, LA",2579006.0
6865,2018,MO,"Jefferson City, MO",286021.0
6866,2018,MO,"Jefferson, MO",1143481.0


In [112]:
_crosswalk = pd.read_csv("../location_crosswalk.csv")
_crosswalk[_crosswalk.pv_city.str.contains("Jefferson")]

/var/folders/v1/7rnrxknn3yz8xm8_z6tv0_ym0000gn/T/ipykernel_76845/2158441640.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  _crosswalk = pd.read_csv("../location_crosswalk.csv")


,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
290,46d7f947-cb8e-11eb-9615-121df0c29c1e,21111,Jeffersontown,NaN,KY,NaN,US,38.1942,NaN,-85.5644,NaN,Jefferson,NaN,21.0,21111.0,NaN
1290,a432b75a-cbae-11eb-9615-121df0c29c1e,37009,Jefferson,NaN,NC,NaN,US,36.4202,NaN,-81.4716,NaN,Ashe,NaN,37.0,37009.0,NaN
2126,3d851962-cb8e-11eb-9615-121df0c29c1e,47089,Jefferson City,NaN,TN,NaN,US,36.1044,NaN,-83.5168,NaN,Jefferson,NaN,47.0,47089.0,NaN
2779,3dc3fc1e-cb8e-11eb-9615-121df0c29c1e,18019,Jeffersonville,NaN,IN,NaN,US,38.2770,NaN,-85.7372,NaN,Clark,NaN,18.0,18019.0,NaN
4690,3e5bfb06-cb8e-11eb-9615-121df0c29c1e,13157,Jefferson,NaN,GA,NaN,US,34.1397,NaN,-83.6258,NaN,Jackson,NaN,13.0,13157.0,NaN
5143,e5cde8e6-09bc-11ec-893a-12de62d610b1,22051,Jefferson,NaN,LA,NaN,US,29.6635,NaN,-90.0846,NaN,Jefferson,NaN,22.0,22051.0,NaN
5670,fdad5bc6-cb8e-11eb-9615-121df0c29c1e,29051,Jefferson City,NaN,MO,NaN,US,38.5712,NaN,-92.2116,NaN,Cole,NaN,29.0,29051.0,NaN
6263,bf1c80de-cb8e-11eb-9615-121df0c29c1e,23015,Jefferson,Jefferson,ME,ME,US,44.2068,-69.459200,-69.4528,44.204430,NaN,Lincoln,23.0,NaN,23015.0
6406,c839b6a2-cb8e-11eb-9615-121df0c29c1e,39097,West Jefferson,NaN,OH,NaN,US,39.9431,NaN,-83.3171,NaN,Madison,NaN,39.0,39097.0,NaN
6410,dff1b7b4-cb8e-11eb-9615-121df0c29c1e,36103,Port Jefferson Station,NaN,NY,NaN,US,40.9229,NaN,-73.0656,NaN,Suffolk,NaN,36.0,36103.0,NaN


In [113]:
merged_df3[(merged_df3.GEOID.isnull())]

,Year,State,Org_Name,Award Amount,GEOID,County_Name
11,2019,AL,"Jefferson, AL",2330492.0,NaN,"Jefferson, AL"
13,2019,AL,"Mobile, AL",1518243.0,NaN,"Mobile, AL"
15,2019,AL,"Mobile, AL",2375186.0,NaN,"Mobile, AL"
17,2019,AL,"Montgomery, AL",1665597.0,NaN,"Montgomery, AL"
20,2019,AL,"Tuscaloosa, AL",783595.0,NaN,"Tuscaloosa, AL"
...,...,...,...,...,...,...
15401,2009,WI,"Milwaukee, WI",16890055.0,NaN,"Milwaukee, WI"
15405,2009,WI,"Racine, WI",1962370.0,NaN,"Racine, WI"
15407,2009,WI,"Sheboygan, WI",1072625.0,NaN,"Sheboygan, WI"
15410,2009,WI,"Waukesha, WI",1026659.0,NaN,"Waukesha, WI"


In [114]:
cdbg_funding[cdbg_funding.Org_Name.str.contains("Los Angeles")]

,Year,State,Org_Name,Award Amount
5118,2019,CA,"Los Angeles, CA",22969231.0
5119,2019,CA,"Los Angeles, CA",53358857.0
6346,2018,CA,"Los Angeles, CA",23428902.0
6347,2018,CA,"Los Angeles, CA",53651938.0
7574,2017,CA,"Los Angeles, CA",20779423.0
7575,2017,CA,"Los Angeles, CA",49416902.0
8836,2016,CA,"Los Angeles, CA",20750346.0
8837,2016,CA,"Los Angeles, CA",49744488.0
10102,2015,CA,"Los Angeles, CA",20619939.0
10103,2015,CA,"Los Angeles, CA",49954532.0


In [115]:
crosswalk[crosswalk.Org_Name.str.contains("Los Angeles")]

,GEOID,County_Name,Org_Name
149,6037,"Los Angeles, CA","Los Angeles, CA"
4907,6037,"Los Angeles, CA","West Los Angeles, CA"
30141,6037,"Los Angeles, VA","Los Angeles, VA"
40086,06037,"Los Angeles, CO","Los Angeles, CO"
40821,06037,"Los Angeles, CA","East Los Angeles, CA"
41156,NaN,"Los Angeles, CA","Los Angeles, CA"


In [116]:
#06037

In [117]:
merged_df3[(merged_df3.GEOID.isnull())].head(40)

,Year,State,Org_Name,Award Amount,GEOID,County_Name
11,2019,AL,"Jefferson, AL",2330492.0,NaN,"Jefferson, AL"
13,2019,AL,"Mobile, AL",1518243.0,NaN,"Mobile, AL"
15,2019,AL,"Mobile, AL",2375186.0,NaN,"Mobile, AL"
17,2019,AL,"Montgomery, AL",1665597.0,NaN,"Montgomery, AL"
20,2019,AL,"Tuscaloosa, AL",783595.0,NaN,"Tuscaloosa, AL"
22,2019,AR,"Conway, AR",472390.0,NaN,"Conway, AR"
43,2019,AZ,"Maricopa, AZ",2989846.0,NaN,"Maricopa, AZ"
49,2019,AZ,"Pima, AZ",2945544.0,NaN,"Pima, AZ"
50,2019,AZ,"Pinal, AZ",1301489.0,NaN,"Pinal, AZ"
58,2019,AZ,"Yuma, AZ",843078.0,NaN,"Yuma, AZ"


In [118]:
#This is a lot
merged_df3.loc[merged_df3.County_Name == "Jefferson, AL", 'GEOID'] = 1073
merged_df3.loc[merged_df3.County_Name == "Mobile, AL", 'GEOID'] = 1097
merged_df3.loc[merged_df3.County_Name == "Montgomery, AL", 'GEOID'] = 1101
merged_df3.loc[merged_df3.County_Name == "Tuscaloosa, AL", 'GEOID'] = 1125
merged_df3.loc[merged_df3.County_Name == "Conway, AR", 'GEOID'] = 5029
merged_df3.loc[merged_df3.County_Name == "Maricopa, AZ", 'GEOID'] = 4013
merged_df3.loc[merged_df3.County_Name == "Pima, AZ", 'GEOID'] = 4019
merged_df3.loc[merged_df3.County_Name == "Pinal, AZ", 'GEOID'] = 4021
merged_df3.loc[merged_df3.County_Name == "Yuma, AZ", 'GEOID'] = 4027
merged_df3.loc[merged_df3.County_Name == "Alameda, CA", 'GEOID'] = 6001
merged_df3.loc[merged_df3.County_Name == "Contra Costa, CA", 'GEOID'] = 6013
merged_df3.loc[merged_df3.County_Name == "Kern, CA", 'GEOID'] = 6029
merged_df3.loc[merged_df3.County_Name == "Fresno, CA", 'GEOID'] = 6019
merged_df3.loc[merged_df3.County_Name == "Los Angeles, CA", 'GEOID'] = 6037
merged_df3.loc[merged_df3.County_Name == "Madera, CA", 'GEOID'] = 6039
merged_df3.loc[merged_df3.County_Name == "Marin, CA", 'GEOID'] = 6041
merged_df3.loc[merged_df3.County_Name == "Merced, CA", 'GEOID'] = 6047
merged_df3.loc[merged_df3.County_Name == "Monterey, CA", 'GEOID'] = 6053
merged_df3.loc[merged_df3.County_Name == "Napa, CA", 'GEOID'] = 6055
merged_df3.loc[merged_df3.County_Name == "Orange, CA", 'GEOID'] = 6059
merged_df3.loc[merged_df3.County_Name == "Riverside, CA", 'GEOID'] = 6065
merged_df3.loc[merged_df3.County_Name == "Sacramento, CA", 'GEOID'] = 6067
merged_df3.loc[merged_df3.County_Name == "San Bernardino, CA", 'GEOID'] = 6071
merged_df3.loc[merged_df3.County_Name == "Ventura, CA", 'GEOID'] = 6111
merged_df3.loc[merged_df3.County_Name == "San Mateo, CA", 'GEOID'] = 6081
merged_df3.loc[merged_df3.County_Name == "San Diego, CA", 'GEOID'] = 6073
merged_df3.loc[merged_df3.County_Name == "San Joaquin, CA", 'GEOID'] = 6077
merged_df3.loc[merged_df3.County_Name == "San Francisco, CA", 'GEOID'] = 6075
merged_df3.loc[merged_df3.County_Name == "San Luis Obispo, CA", 'GEOID'] = 6079
merged_df3.loc[merged_df3.County_Name == "Santa Barbara, CA", 'GEOID'] = 6083
merged_df3.loc[merged_df3.County_Name == "Santa Clara, CA", 'GEOID'] = 6085
merged_df3.loc[merged_df3.County_Name == "Santa Cruz, CA", 'GEOID'] = 6087
merged_df3.loc[merged_df3.County_Name == "Sonoma, CA", 'GEOID'] = 6097
merged_df3.loc[merged_df3.County_Name == "Stanislaus, CA", 'GEOID'] = 6099
merged_df3.loc[merged_df3.County_Name == "Tulare, CA", 'GEOID'] = 6107
merged_df3.loc[merged_df3.County_Name == "Adams, CO", 'GEOID'] = 8001
merged_df3.loc[merged_df3.County_Name == "Arapahoe, CO", 'GEOID'] = 8005
merged_df3.loc[merged_df3.County_Name == "Boulder, CO", 'GEOID'] = 8013
merged_df3.loc[merged_df3.County_Name == "Broomfield, CO", 'GEOID'] = 8014
merged_df3.loc[merged_df3.County_Name == "Denver, CO", 'GEOID'] = 8031
merged_df3.loc[merged_df3.County_Name == "El Paso, CO", 'GEOID'] = 8041
merged_df3.loc[merged_df3.County_Name == "Jefferson, CO", 'GEOID'] = 8059
merged_df3.loc[merged_df3.County_Name == "Pueblo, CO", 'GEOID'] = 8101
merged_df3.loc[merged_df3.County_Name == "Fairfield, CT", 'GEOID'] = 9001
merged_df3.loc[merged_df3.County_Name == "Hartford, CT", 'GEOID'] = 9003
merged_df3.loc[merged_df3.County_Name == "New Haven, CT", 'GEOID'] = 9009
merged_df3.loc[merged_df3.County_Name == "New London, CT", 'GEOID'] = 9011
merged_df3.loc[merged_df3.County_Name == "New Castle, DE", 'GEOID'] = 10003
merged_df3.loc[merged_df3.County_Name == "Brevard, FL", 'GEOID'] = 12009
merged_df3.loc[merged_df3.County_Name == "Broward, FL", 'GEOID'] = 12011
merged_df3.loc[merged_df3.County_Name == "Collier, FL", 'GEOID'] = 12021
merged_df3.loc[merged_df3.County_Name == "Escambia, FL", 'GEOID'] = 12033
merged_df3.loc[merged_df3.County_Name == "Flagler, FL", 'GEOID'] = 12035
merged_df3.loc[merged_df3.County_Name == "Hillsborough, FL", 'GEOID'] = 12057
merged_df3.loc[merged_df3.County_Name == "Lake, FL", 'GEOID'] = 12069
merged_df3.loc[merged_df3.County_Name == "Lee, FL", 'GEOID'] = 12071
merged_df3.loc[merged_df3.County_Name == "Manatee, FL", 'GEOID'] = 12081
merged_df3.loc[merged_df3.County_Name == "Marion, FL", 'GEOID'] = 12083
merged_df3.loc[merged_df3.County_Name == "Miami-Dade, FL", 'GEOID'] = 12086
merged_df3.loc[merged_df3.County_Name == "Orange, FL", 'GEOID'] = 12095
merged_df3.loc[merged_df3.County_Name == "Osceola, FL", 'GEOID'] = 12097
merged_df3.loc[merged_df3.County_Name == "Palm Beach, FL", 'GEOID'] = 12099
merged_df3.loc[merged_df3.County_Name == "Pasco, FL", 'GEOID'] = 12101
merged_df3.loc[merged_df3.County_Name == "Pinellas, FL", 'GEOID'] = 12103
merged_df3.loc[merged_df3.County_Name == "Polk, FL", 'GEOID'] = 12105
merged_df3.loc[merged_df3.County_Name == "St. Johns, FL", 'GEOID'] = 12109
merged_df3.loc[merged_df3.County_Name == "Sarasota, FL", 'GEOID'] = 12115
merged_df3.loc[merged_df3.County_Name == "Seminole, FL", 'GEOID'] = 12117
merged_df3.loc[merged_df3.County_Name == "Volusia, FL", 'GEOID'] = 12127
merged_df3.loc[merged_df3.County_Name == "Cherokee, GA", 'GEOID'] = 13057
merged_df3.loc[merged_df3.County_Name == "Clayton, GA", 'GEOID'] = 13063
merged_df3.loc[merged_df3.County_Name == "Cobb, GA", 'GEOID'] = 13067
merged_df3.loc[merged_df3.County_Name == "DeKalb, GA", 'GEOID'] = 13089
merged_df3.loc[merged_df3.County_Name == "Fulton, GA", 'GEOID'] = 13121
merged_df3.loc[merged_df3.County_Name == "Gwinnett, GA", 'GEOID'] = 13135
merged_df3.loc[merged_df3.County_Name == "Henry, GA", 'GEOID'] = 13151
merged_df3.loc[merged_df3.County_Name == "Macon, GA", 'GEOID'] = 13193
merged_df3.loc[merged_df3.County_Name == "Hawaii, HI", 'GEOID'] = 15001
merged_df3.loc[merged_df3.County_Name == "Honolulu, HI", 'GEOID'] = 15003
merged_df3.loc[merged_df3.County_Name == "Kauai, HI", 'GEOID'] = 15007
merged_df3.loc[merged_df3.County_Name == "Maui, HI", 'GEOID'] = 15009
merged_df3.loc[merged_df3.County_Name == "Des Moines, IA", 'GEOID'] = 19057
merged_df3.loc[merged_df3.County_Name == "Dubuque, IA", 'GEOID'] = 19061
merged_df3.loc[merged_df3.County_Name == "Boise, ID", 'GEOID'] = 16015
merged_df3.loc[merged_df3.County_Name == "Kootenai, ID", 'GEOID'] = 16055
merged_df3.loc[merged_df3.County_Name == "Champaign, IL", 'GEOID'] = 17021
merged_df3.loc[merged_df3.County_Name == "Cook, IL", 'GEOID'] = 17031
merged_df3.loc[merged_df3.County_Name == "DeKalb, IL", 'GEOID'] = 17037
merged_df3.loc[merged_df3.County_Name == "DuPage, IL", 'GEOID'] = 17043
merged_df3.loc[merged_df3.County_Name == "Kane, IL", 'GEOID'] = 17089
merged_df3.loc[merged_df3.County_Name == "Kankakee, IL", 'GEOID'] = 17091
merged_df3.loc[merged_df3.County_Name == "Lake, IL", 'GEOID'] = 17097
merged_df3.loc[merged_df3.County_Name == "Madison, IL", 'GEOID'] = 17119
merged_df3.loc[merged_df3.County_Name == "McHenry, IL", 'GEOID'] = 17111
merged_df3.loc[merged_df3.County_Name == "Peoria, IL", 'GEOID'] = 17143
merged_df3.loc[merged_df3.County_Name == "Rock Island, IL", 'GEOID'] = 17161
merged_df3.loc[merged_df3.County_Name == "St. Clair, IL", 'GEOID'] = 17163
merged_df3.loc[merged_df3.County_Name == "Will, IL", 'GEOID'] = 17197
merged_df3.loc[merged_df3.County_Name == "Elkhart, IN", 'GEOID'] = 18039
merged_df3.loc[merged_df3.County_Name == "Hamilton, IN", 'GEOID'] = 18057
merged_df3.loc[merged_df3.County_Name == "Lake, IN", 'GEOID'] = 18089
merged_df3.loc[merged_df3.County_Name == "Johnson, KS", 'GEOID'] = 20091
merged_df3.loc[merged_df3.County_Name == "Leavenworth, KS", 'GEOID'] = 20103
merged_df3.loc[merged_df3.County_Name == "Shawnee, KS", 'GEOID'] = 20177
merged_df3.loc[merged_df3.County_Name == "Wichita, KS", 'GEOID'] = 20203
merged_df3.loc[merged_df3.County_Name == "Henderson, KY", 'GEOID'] = 21101
merged_df3.loc[merged_df3.County_Name == "Jefferson, LA", 'GEOID'] = 22051
merged_df3.loc[merged_df3.County_Name == "Lafayette, LA", 'GEOID'] = 22055
merged_df3.loc[merged_df3.County_Name == "St. Tammany, LA", 'GEOID'] = 22103
merged_df3.loc[merged_df3.County_Name == "Barnstable, MA", 'GEOID'] = 25001
merged_df3.loc[merged_df3.County_Name == "Plymouth, MA", 'GEOID'] = 25023
merged_df3.loc[merged_df3.County_Name == "Worcester, MA", 'GEOID'] = 25027
merged_df3.loc[merged_df3.County_Name == "Anne Arundel, MD", 'GEOID'] = 24003
merged_df3.loc[merged_df3.County_Name == "Baltimore, MD", 'GEOID'] = 24005
merged_df3.loc[merged_df3.County_Name == "Frederick, MD", 'GEOID'] = 24021
merged_df3.loc[merged_df3.County_Name == "Harford, MD", 'GEOID'] = 24025
merged_df3.loc[merged_df3.County_Name == "Howard, MD", 'GEOID'] = 24027
merged_df3.loc[merged_df3.County_Name == "Montgomery, MD", 'GEOID'] = 24031
merged_df3.loc[merged_df3.County_Name == "Prince George's, MD", 'GEOID'] = 24033
merged_df3.loc[merged_df3.County_Name == "Cumberland, ME", 'GEOID'] = 23005
merged_df3.loc[merged_df3.County_Name == "Clinton, MI", 'GEOID'] = 26037
merged_df3.loc[merged_df3.County_Name == "Genesee, MI", 'GEOID'] = 26047
merged_df3.loc[merged_df3.County_Name == "Jackson, MI", 'GEOID'] = 26075
merged_df3.loc[merged_df3.County_Name == "Kalamazoo, MI", 'GEOID'] = 26077
merged_df3.loc[merged_df3.County_Name == "Kent, MI", 'GEOID'] = 26081
merged_df3.loc[merged_df3.County_Name == "Macomb, MI", 'GEOID'] = 26099
merged_df3.loc[merged_df3.County_Name == "Midland, MI", 'GEOID'] = 26111
merged_df3.loc[merged_df3.County_Name == "Monroe, MI", 'GEOID'] = 26115
merged_df3.loc[merged_df3.County_Name == "Muskegon, MI", 'GEOID'] = 26121
merged_df3.loc[merged_df3.County_Name == "Oakland, MI", 'GEOID'] = 26125
merged_df3.loc[merged_df3.County_Name == "Saginaw, MI", 'GEOID'] = 26145
merged_df3.loc[merged_df3.County_Name == "Washtenaw, MI", 'GEOID'] = 26161
merged_df3.loc[merged_df3.County_Name == "Wayne, MI", 'GEOID'] = 26163
merged_df3.loc[merged_df3.County_Name == "Anoka, MN", 'GEOID'] = 27003
merged_df3.loc[merged_df3.County_Name == "Dakota, MN", 'GEOID'] = 27037
merged_df3.loc[merged_df3.County_Name == "Hennepin, MN", 'GEOID'] = 27053
merged_df3.loc[merged_df3.County_Name == "Ramsey, MN", 'GEOID'] = 27123
merged_df3.loc[merged_df3.County_Name == "St. Louis, MN", 'GEOID'] = 27137
merged_df3.loc[merged_df3.County_Name == "Washington, MN", 'GEOID'] = 27163
merged_df3.loc[merged_df3.County_Name == "Jefferson, MO", 'GEOID'] = 29099
merged_df3.loc[merged_df3.County_Name == "St. Charles, MO", 'GEOID'] = 29183
merged_df3.loc[merged_df3.County_Name == "St. Louis, MO", 'GEOID'] = 29189
merged_df3.loc[merged_df3.County_Name == "Jackson, MS", 'GEOID'] = 28057
merged_df3.loc[merged_df3.County_Name == "Missoula, MT", 'GEOID'] = 30063
merged_df3.loc[merged_df3.County_Name == "Cumberland, NC", 'GEOID'] = 37051
merged_df3.loc[merged_df3.County_Name == "Durham, NC", 'GEOID'] = 37063
merged_df3.loc[merged_df3.County_Name == "Lenoir, NC", 'GEOID'] = 37107
merged_df3.loc[merged_df3.County_Name == "Mecklenburg, NC", 'GEOID'] = 37119
merged_df3.loc[merged_df3.County_Name == "Union, NC", 'GEOID'] = 37179
merged_df3.loc[merged_df3.County_Name == "Wake, NC", 'GEOID'] = 37183
merged_df3.loc[merged_df3.County_Name == "Grand Forks, ND", 'GEOID'] = 38035
merged_df3.loc[merged_df3.County_Name == "Lincoln, NE", 'GEOID'] = 31111
merged_df3.loc[merged_df3.County_Name == "Atlantic, NJ", 'GEOID'] = 34001
merged_df3.loc[merged_df3.County_Name == "Bergen, NJ", 'GEOID'] = 34003
merged_df3.loc[merged_df3.County_Name == "Burlington, NJ", 'GEOID'] = 34005
merged_df3.loc[merged_df3.County_Name == "Camden, NJ", 'GEOID'] = 34007
merged_df3.loc[merged_df3.County_Name == "Essex, NJ", 'GEOID'] = 34013
merged_df3.loc[merged_df3.County_Name == "Gloucester, NJ", 'GEOID'] = 34015
merged_df3.loc[merged_df3.County_Name == "Hudson, NJ", 'GEOID'] = 34017
merged_df3.loc[merged_df3.County_Name == "Middlesex, NJ", 'GEOID'] = 34023
merged_df3.loc[merged_df3.County_Name == "Monmouth, NJ", 'GEOID'] = 34025
merged_df3.loc[merged_df3.County_Name == "Morris, NJ", 'GEOID'] = 34027
merged_df3.loc[merged_df3.County_Name == "Ocean, NJ", 'GEOID'] = 34029
merged_df3.loc[merged_df3.County_Name == "Passaic, NJ", 'GEOID'] = 34031
merged_df3.loc[merged_df3.County_Name == "Somerset, NJ", 'GEOID'] = 34035
merged_df3.loc[merged_df3.County_Name == "Union, NJ", 'GEOID'] = 34039
merged_df3.loc[merged_df3.County_Name == "Santa Fe, NM", 'GEOID'] = 35049
merged_df3.loc[merged_df3.County_Name == "Clark, NV", 'GEOID'] = 32003
merged_df3.loc[merged_df3.County_Name == "Albany, NY", 'GEOID'] = 36001
merged_df3.loc[merged_df3.County_Name == "Dutchess, NY", 'GEOID'] = 36027
merged_df3.loc[merged_df3.County_Name == "Erie, NY", 'GEOID'] = 36029
merged_df3.loc[merged_df3.County_Name == "Monroe, NY", 'GEOID'] = 36055
merged_df3.loc[merged_df3.County_Name == "Nassau, NY", 'GEOID'] = 36059
merged_df3.loc[merged_df3.County_Name == "New York, NY", 'GEOID'] = 36061
merged_df3.loc[merged_df3.County_Name == "Onondaga, NY", 'GEOID'] = 36067
merged_df3.loc[merged_df3.County_Name == "Orange, NY", 'GEOID'] = 36071
merged_df3.loc[merged_df3.County_Name == "Rockland, NY", 'GEOID'] = 36087
merged_df3.loc[merged_df3.County_Name == "Schenectady, NY", 'GEOID'] = 36093
merged_df3.loc[merged_df3.County_Name == "Suffolk, NY", 'GEOID'] = 36103
merged_df3.loc[merged_df3.County_Name == "Westchester, NY", 'GEOID'] = 36119
merged_df3.loc[merged_df3.County_Name == "Butler, OH", 'GEOID'] = 39017
merged_df3.loc[merged_df3.County_Name == "Clermont, OH", 'GEOID'] = 39025
merged_df3.loc[merged_df3.County_Name == "Cuyahoga, OH", 'GEOID'] = 39035
merged_df3.loc[merged_df3.County_Name == "Franklin, OH", 'GEOID'] = 39049
merged_df3.loc[merged_df3.County_Name == "Hamilton, OH", 'GEOID'] = 39061
merged_df3.loc[merged_df3.County_Name == "Lake, OH", 'GEOID'] = 39085
merged_df3.loc[merged_df3.County_Name == "Lorain, OH", 'GEOID'] = 39093
merged_df3.loc[merged_df3.County_Name == "Montgomery, OH", 'GEOID'] = 39113
merged_df3.loc[merged_df3.County_Name == "Sandusky, OH", 'GEOID'] = 39143
merged_df3.loc[merged_df3.County_Name == "Stark, OH", 'GEOID'] = 39151
merged_df3.loc[merged_df3.County_Name == "Summit, OH", 'GEOID'] = 39153
merged_df3.loc[merged_df3.County_Name == "Warren, OH", 'GEOID'] = 39165
merged_df3.loc[merged_df3.County_Name == "Tulsa, OK", 'GEOID'] = 40143
merged_df3.loc[merged_df3.County_Name == "Clackamas, OR", 'GEOID'] = 41005
merged_df3.loc[merged_df3.County_Name == "Multnomah, OR", 'GEOID'] = 41051
merged_df3.loc[merged_df3.County_Name == "Washington, OR", 'GEOID'] = 41067
merged_df3.loc[merged_df3.County_Name == "Allegheny, PA", 'GEOID'] = 42003
merged_df3.loc[merged_df3.County_Name == "Beaver, PA", 'GEOID'] = 42007
merged_df3.loc[merged_df3.County_Name == "Berks, PA", 'GEOID'] = 42011
merged_df3.loc[merged_df3.County_Name == "Bucks, PA", 'GEOID'] = 42017
merged_df3.loc[merged_df3.County_Name == "Chester, PA", 'GEOID'] = 42029
merged_df3.loc[merged_df3.County_Name == "Cumberland, PA", 'GEOID'] = 42041
merged_df3.loc[merged_df3.County_Name == "Dauphin, PA", 'GEOID'] = 42043
merged_df3.loc[merged_df3.County_Name == "Delaware, PA", 'GEOID'] = 42045
merged_df3.loc[merged_df3.County_Name == "Erie, PA", 'GEOID'] = 42049
merged_df3.loc[merged_df3.County_Name == "Lancaster, PA", 'GEOID'] = 42071
merged_df3.loc[merged_df3.County_Name == "Lebanon, PA", 'GEOID'] = 42075
merged_df3.loc[merged_df3.County_Name == "Lehigh, PA", 'GEOID'] = 42077
merged_df3.loc[merged_df3.County_Name == "Luzerne, PA", 'GEOID'] = 42079
merged_df3.loc[merged_df3.County_Name == "Montgomery, PA", 'GEOID'] = 42091
merged_df3.loc[merged_df3.County_Name == "Northampton, PA", 'GEOID'] = 42095
merged_df3.loc[merged_df3.County_Name == "Philadelphia, PA", 'GEOID'] = 42101
merged_df3.loc[merged_df3.County_Name == "Washington, PA", 'GEOID'] = 42125
merged_df3.loc[merged_df3.County_Name == "Westmoreland, PA", 'GEOID'] = 42129
merged_df3.loc[merged_df3.County_Name == "York, PA", 'GEOID'] = 42133
merged_df3.loc[merged_df3.County_Name == "Providence, RI", 'GEOID'] = 44007
merged_df3.loc[merged_df3.County_Name == "Aiken, SC", 'GEOID'] = 45003
merged_df3.loc[merged_df3.County_Name == "Anderson, SC", 'GEOID'] = 45007
merged_df3.loc[merged_df3.County_Name == "Charleston, SC", 'GEOID'] = 45019
merged_df3.loc[merged_df3.County_Name == "Florence, SC", 'GEOID'] = 45041
merged_df3.loc[merged_df3.County_Name == "Greenville, SC", 'GEOID'] = 45045
merged_df3.loc[merged_df3.County_Name == "Horry, SC", 'GEOID'] = 45051
merged_df3.loc[merged_df3.County_Name == "Lexington, SC", 'GEOID'] = 45063
merged_df3.loc[merged_df3.County_Name == "Richland, SC", 'GEOID'] = 45079
merged_df3.loc[merged_df3.County_Name == "Spartanburg, SC", 'GEOID'] = 45083
merged_df3.loc[merged_df3.County_Name == "Sumter, SC", 'GEOID'] = 45083
merged_df3.loc[merged_df3.County_Name == "Franklin, TN", 'GEOID'] = 47051
merged_df3.loc[merged_df3.County_Name == "Jackson, TN", 'GEOID'] = 47087
merged_df3.loc[merged_df3.County_Name == "Knox, TN", 'GEOID'] = 47093
merged_df3.loc[merged_df3.County_Name == "Shelby, TN", 'GEOID'] = 47157
merged_df3.loc[merged_df3.County_Name == "Austin, TX", 'GEOID'] = 48015
merged_df3.loc[merged_df3.County_Name == "Bexar, TX", 'GEOID'] = 48029
merged_df3.loc[merged_df3.County_Name == "Brazoria, TX", 'GEOID'] = 48039
merged_df3.loc[merged_df3.County_Name == "Dallas, TX", 'GEOID'] = 48113
merged_df3.loc[merged_df3.County_Name == "Denton, TX", 'GEOID'] = 48121
merged_df3.loc[merged_df3.County_Name == "El Paso, TX", 'GEOID'] = 48141
merged_df3.loc[merged_df3.County_Name == "Fort Bend, TX", 'GEOID'] = 48157
merged_df3.loc[merged_df3.County_Name == "Galveston, TX", 'GEOID'] = 48167
merged_df3.loc[merged_df3.County_Name == "Harris, TX", 'GEOID'] = 48201
merged_df3.loc[merged_df3.County_Name == "Hidalgo, TX", 'GEOID'] = 48215
merged_df3.loc[merged_df3.County_Name == "Houston, TX", 'GEOID'] = 48225
merged_df3.loc[merged_df3.County_Name == "Lubbock, TX", 'GEOID'] = 48303
merged_df3.loc[merged_df3.County_Name == "Midland, TX", 'GEOID'] = 48329
merged_df3.loc[merged_df3.County_Name == "Montgomery, TX", 'GEOID'] = 48339
merged_df3.loc[merged_df3.County_Name == "Orange, TX", 'GEOID'] = 48361
merged_df3.loc[merged_df3.County_Name == "Sherman, TX", 'GEOID'] = 48421
merged_df3.loc[merged_df3.County_Name == "Tarrant, TX", 'GEOID'] = 48439
merged_df3.loc[merged_df3.County_Name == "Travis, TX", 'GEOID'] = 48453
merged_df3.loc[merged_df3.County_Name == "Tyler, TX", 'GEOID'] = 48457
merged_df3.loc[merged_df3.County_Name == "Victoria, TX", 'GEOID'] = 48469
merged_df3.loc[merged_df3.County_Name == "Williamson, TX", 'GEOID'] = 48491
merged_df3.loc[merged_df3.County_Name == "Davis, UT", 'GEOID'] = 49011
merged_df3.loc[merged_df3.County_Name == "Salt Lake, UT", 'GEOID'] = 49035
merged_df3.loc[merged_df3.County_Name == "Utah, UT", 'GEOID'] = 49049
merged_df3.loc[merged_df3.County_Name == "Arlington, VA", 'GEOID'] = 51013
merged_df3.loc[merged_df3.County_Name == "Chesterfield, VA", 'GEOID'] = 51041
merged_df3.loc[merged_df3.County_Name == "Fairfax, VA", 'GEOID'] = 51059
merged_df3.loc[merged_df3.County_Name == "Henrico, VA", 'GEOID'] = 51087
merged_df3.loc[merged_df3.County_Name == "Loudoun, VA", 'GEOID'] = 51107
merged_df3.loc[merged_df3.County_Name == "Prince William, VA", 'GEOID'] = 51153
merged_df3.loc[merged_df3.County_Name == "Richmond, VA", 'GEOID'] = 51159
merged_df3.loc[merged_df3.County_Name == "Roanoke, VA", 'GEOID'] = 51161
merged_df3.loc[merged_df3.County_Name == "Clark, WA", 'GEOID'] = 53011
merged_df3.loc[merged_df3.County_Name == "King, WA", 'GEOID'] = 53033
merged_df3.loc[merged_df3.County_Name == "Kitsap, WA", 'GEOID'] = 53035
merged_df3.loc[merged_df3.County_Name == "Pierce, WA", 'GEOID'] = 53053
merged_df3.loc[merged_df3.County_Name == "Snohomish, WA", 'GEOID'] = 53061
merged_df3.loc[merged_df3.County_Name == "Spokane, WA", 'GEOID'] = 53063
merged_df3.loc[merged_df3.County_Name == "Thurston, WA", 'GEOID'] = 53067
merged_df3.loc[merged_df3.County_Name == "Walla Walla, WA", 'GEOID'] = 53071
merged_df3.loc[merged_df3.County_Name == "Yakima, WA", 'GEOID'] = 53077
merged_df3.loc[merged_df3.County_Name == "Dane, WI", 'GEOID'] = 55025
merged_df3.loc[merged_df3.County_Name == "Eau Claire, WI", 'GEOID'] = 55035
merged_df3.loc[merged_df3.County_Name == "Fond du Lac, WI", 'GEOID'] = 55039
merged_df3.loc[merged_df3.County_Name == "Kenosha, WI", 'GEOID'] = 55059
merged_df3.loc[merged_df3.County_Name == "La Crosse, WI", 'GEOID'] = 55063
merged_df3.loc[merged_df3.County_Name == "Milwaukee, WI", 'GEOID'] = 55079
merged_df3.loc[merged_df3.County_Name == "Racine, WI", 'GEOID'] = 55101
merged_df3.loc[merged_df3.County_Name == "Sheboygan, WI", 'GEOID'] = 55117
merged_df3.loc[merged_df3.County_Name == "Waukesha, WI", 'GEOID'] = 55133
merged_df3.loc[merged_df3.County_Name == "Douglas, CO", 'GEOID'] = 8035
merged_df3.loc[merged_df3.County_Name == "Carson City, NV", 'GEOID'] = 32510
merged_df3.loc[merged_df3.County_Name == "Marion, IL", 'GEOID'] = 17121

In [119]:
merged_df3[(merged_df3.GEOID.isnull())]

,Year,State,Org_Name,Award Amount,GEOID,County_Name


In [120]:
grouped = merged_df3.groupby(['GEOID', 'County_Name', 'Year']).sum('Award Amount')
grouped

Award Amount
GEOID County_Name    Year              
1003  Baldwin, AL    2015       65077.0
                     2016       66961.0
                     2017       80121.0
                     2018       94768.0
                     2019       93529.0
...                                 ...
56025 Natrona, WY    2018      368809.0
37161 Rutherford, NC 2016      759810.0
                     2017      794592.0
                     2018      873769.0
                     2019      834665.0

[6630 rows x 1 columns]

In [121]:
grouped = grouped.reset_index()
grouped.head()

,GEOID,County_Name,Year,Award Amount
0,1003,"Baldwin, AL",2015,65077.0
1,1003,"Baldwin, AL",2016,66961.0
2,1003,"Baldwin, AL",2017,80121.0
3,1003,"Baldwin, AL",2018,94768.0
4,1003,"Baldwin, AL",2019,93529.0


Need to address independent cities

In [122]:
grouped[grouped["County_Name"].str.contains("Baltimore")]

,GEOID,County_Name,Year,Award Amount
2510,24005,"Baltimore, MD",2009,27408336.0
2511,24005,"Baltimore, MD",2010,29678888.0
2512,24005,"Baltimore, MD",2011,24787088.0
2513,24005,"Baltimore, MD",2012,21461386.0
2514,24005,"Baltimore, MD",2013,22575299.0
2515,24005,"Baltimore, MD",2014,22521940.0
2516,24005,"Baltimore, MD",2015,22381983.0
2517,24005,"Baltimore, MD",2016,22703846.0
2518,24005,"Baltimore, MD",2017,22973149.0
2519,24005,"Baltimore, MD",2018,25475654.0


In [123]:
merged_df2.columns

Index(['GeoFIPS', 'GeoName', 'Earnings by place of work',
       'Employee and self-employed contributions for government social insurance',
       'Employer contributions for employee pension and insurance funds 8',
       'Employer contributions for government social insurance',
       'Equals: Net earnings by place of residence', 'Farm income 2',
       'Farm proprietors' income',
       'Less: Contributions for government social insurance 5',
       'Nonfarm personal income 1', 'Nonfarm proprietors' income',
       'Per capita personal income (dollars) 4',
       'Personal income (thousands of dollars)',
       'Plus: Adjustment for residence 6',
       'Plus: Dividends, interest, and rent 7',
       'Plus: Personal current transfer receipts', 'Population (persons) 3',
       'Proprietors employment', 'Proprietors' income 9',
       'Supplements to wages and salaries', 'Total employment',
       'Wage and salary employment', 'Wages and salaries', 'year', 'STNAME',
       'CTYNAME',

In [124]:
grouped["County Code"] = grouped["GEOID"]

In [125]:
merged_df4 = pd.merge(merged_df2, grouped, how="left", on=["County Code"])
merged_df4

,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,State,County,County Code,fipstate,fipscty,Applications,GEOID,County_Name,Year,Award Amount
0,"""01001""","Autauga, AL",535644,39092,69250,28256,1160479,11424,8731,67348,...,AL,Autauga County,1001,1,1,288,NaN,NaN,NaN,NaN
1,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,AL,Baldwin County,1003,1,3,1505,1003,"Baldwin, AL",2015.0,65077.0
2,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,AL,Baldwin County,1003,1,3,1505,1003,"Baldwin, AL",2016.0,66961.0
3,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,AL,Baldwin County,1003,1,3,1505,1003,"Baldwin, AL",2017.0,80121.0
4,"""01003""","Baldwin, AL",2721141,196160,330994,147843,3625277,14632,1012,344003,...,AL,Baldwin County,1003,1,3,1505,1003,"Baldwin, AL",2018.0,94768.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99094,"""56037""","Sweetwater, WY",2186822,122914,243513,125358,1778004,9846,8485,248272,...,WY,Sweetwater County,56037,56,37,416,NaN,NaN,NaN,NaN
99095,"""56039""","Teton, WY",1858314,102201,137206,107871,1519128,3364,-483,210072,...,WY,Teton County,56039,56,39,1817,NaN,NaN,NaN,NaN
99096,"""56041""","Uinta, WY",546044,34219,79408,36393,536355,4720,2823,70612,...,WY,Uinta County,56041,56,41,190,NaN,NaN,NaN,NaN
99097,"""56043""","Washakie, WY",255080,15662,33784,15405,218093,8598,4675,31067,...,WY,Washakie County,56043,56,43,96,NaN,NaN,NaN,NaN


In [126]:
merged_df4[merged_df4.GEOID.isnull()]

,GeoFIPS,GeoName,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,State,County,County Code,fipstate,fipscty,Applications,GEOID,County_Name,Year,Award Amount
0,"""01001""","Autauga, AL",535644,39092,69250,28256,1160479,11424,8731,67348,...,AL,Autauga County,1001,1,1,288,NaN,NaN,NaN,NaN
6,"""01005""","Barbour, AL",429668,28956,64008,23435,387341,18712,15908,52391,...,AL,Barbour County,1005,1,5,133,NaN,NaN,NaN,NaN
7,"""01007""","Bibb, AL",206853,15973,29693,10729,321855,2655,2515,26702,...,AL,Bibb County,1007,1,7,98,NaN,NaN,NaN,NaN
8,"""01009""","Blount, AL",462562,36182,56706,22040,993044,20257,16640,58222,...,AL,Blount County,1009,1,9,247,NaN,NaN,NaN,NaN
9,"""01011""","Bullock, AL",135903,8373,19881,6779,137654,16503,11354,15152,...,AL,Bullock County,1011,1,11,31,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99094,"""56037""","Sweetwater, WY",2186822,122914,243513,125358,1778004,9846,8485,248272,...,WY,Sweetwater County,56037,56,37,416,NaN,NaN,NaN,NaN
99095,"""56039""","Teton, WY",1858314,102201,137206,107871,1519128,3364,-483,210072,...,WY,Teton County,56039,56,39,1817,NaN,NaN,NaN,NaN
99096,"""56041""","Uinta, WY",546044,34219,79408,36393,536355,4720,2823,70612,...,WY,Uinta County,56041,56,41,190,NaN,NaN,NaN,NaN
99097,"""56043""","Washakie, WY",255080,15662,33784,15405,218093,8598,4675,31067,...,WY,Washakie County,56043,56,43,96,NaN,NaN,NaN,NaN


In [127]:
cdbg_funding[cdbg_funding.Org_Name.str.contains("Barbour")]

,Year,State,Org_Name,Award Amount


In [128]:
merged_df4.to_csv("ei_final.csv")